In [2]:
import xml.etree.ElementTree as ET
import re
import unicodedata
from docx import Document
from lxml import etree
import pandas as pd
import os
from win32com import client


In [31]:
from lxml import etree

def convert_custom_xml_to_tei(input_file, output_file, appid=3):
    # Define TEI and XML namespaces
    tei_ns = "http://www.tei-c.org/ns/1.0"
    xml_ns = "http://www.w3.org/XML/1998/namespace"

    # Roman numeral mapping for appid conversion in references
    roman_numerals = {
        "I": "1", "II": "2", "III": "3", "IV": "4", "V": "5", "VI": "6"
    }

    special_sigla = {"(!)": "Uncertain reading", "..": "Uncertain letters"}

    # Parse input XML
    tree = etree.parse(input_file)
    root = tree.getroot()

    # Create TEI root
    tei = etree.Element(f"{{{tei_ns}}}TEI", nsmap={None: tei_ns, "xml": xml_ns})
    
    # Add TEI Header
    teiHeader = etree.SubElement(tei, f"{{{tei_ns}}}teiHeader")
    fileDesc = etree.SubElement(teiHeader, f"{{{tei_ns}}}fileDesc")
    titleStmt = etree.SubElement(fileDesc, f"{{{tei_ns}}}titleStmt")
    title = etree.SubElement(titleStmt, f"{{{tei_ns}}}title")
    title.text = "Converted TEI Document"
    pubStmt = etree.SubElement(fileDesc, f"{{{tei_ns}}}publicationStmt")
    pubStmt.text = "Generated from custom XML"
    sourceDesc = etree.SubElement(fileDesc, f"{{{tei_ns}}}sourceDesc")
    sourceDesc.text = "Converted from custom format"

    # Add <text> structure
    text = etree.SubElement(tei, f"{{{tei_ns}}}text")
    body = etree.SubElement(text, f"{{{tei_ns}}}body")
    listApp = etree.SubElement(body, f"{{{tei_ns}}}listApp")
    # Create a witness list section at the beginning
    listWit = etree.SubElement(tei, f"{{{tei_ns}}}listWit")
    wit_mapping = {}  # Dictionary to store id-to-name mapping
    # Create a section to store sigla explanations
    listPrefixDef = etree.SubElement(tei, f"{{{tei_ns}}}listPrefixDef")
    sigla_mapping = {}  # Dictionary to store id-to-explanation mapping
    # Define abbreviations for sigla
    sigla_id_mapping = {
        "(!)": "sig-unc",
        "..": "sig-ell"
    }
    
    
    # Process <entry> elements
    for entry in root.findall("entry"):
        entry_number = entry.get("Number")
        chapter = entry.findtext("LemmaInfo/Chapter")
        verse_from = entry.findtext("LemmaInfo/Verses/From")
        verse_to = entry.findtext("LemmaInfo/Verses/To")
        verse = verse_from if verse_from else entry.findtext("LemmaInfo/Verses")

        
        # Ensure values exist
        chapter = chapter if chapter else "unknown_chap"
        entry_number = entry_number if entry_number else "unknown_entry"

        # Define loc, from, and to attributes properly
        app_attributes = {f"{{{xml_ns}}}id": f"app{appid}-{chapter}-{verse}-{entry_number}"}
        if verse_from:
            app_attributes["from"] = verse_from
        if verse_to:
            app_attributes["to"] = verse_to

        # Create <app> element
        app = etree.SubElement(listApp, f"{{{tei_ns}}}app", app_attributes)

       # Add lemmas
        lemmas_section = entry.find("LemmaInfo/Lemmas")
        if lemmas_section is not None:
            # Check if there is a "From" and "To" section to determine if it's a range
            from_lemmas = lemmas_section.findall("From/Lemma")
            to_lemmas = lemmas_section.findall("To/Lemma")

            if from_lemmas and to_lemmas:
                # Create a lemma group for ranged lemmas
                lemGrp = etree.SubElement(app, f"{{{tei_ns}}}lemGrp", {"type": "range"})

                # Create a "From" lem group
                from_lemGrp = etree.SubElement(lemGrp, f"{{{tei_ns}}}lemGrp", {"type": "from"})
                from_lem = etree.SubElement(from_lemGrp, f"{{{tei_ns}}}lem")

                for lemma in from_lemmas:
                    lemma_text = lemma.findtext("Text")
                    lemma_number = lemma.findtext("Number")

                    if lemma_text:
                        w = etree.SubElement(from_lem, f"{{{tei_ns}}}w")
                        w.text = lemma_text
                        if lemma_number:
                            w.set("n", lemma_number)  # Add @n if a number exists

                # Create a "To" lem group
                to_lemGrp = etree.SubElement(lemGrp, f"{{{tei_ns}}}lemGrp", {"type": "to"})
                to_lem = etree.SubElement(to_lemGrp, f"{{{tei_ns}}}lem")

                for lemma in to_lemmas:
                    lemma_text = lemma.findtext("Text")
                    lemma_number = lemma.findtext("Number")

                    if lemma_text:
                        w = etree.SubElement(to_lem, f"{{{tei_ns}}}w")
                        w.text = lemma_text
                        if lemma_number:
                            w.set("n", lemma_number)  # Add @n if a number exists

            #check if there is k\q
            k_lemma = lemmas_section.find("Lemma[K='True']")
            q_lemma = lemmas_section.find("Lemma[Q='True']")

            # If both K and Q exist, create an <rdgGrp type="k/q">
            if k_lemma is not None and q_lemma is not None:
                kq_rdgGrp = etree.SubElement(app, f"{{{tei_ns}}}rdgGrp", {"type": "k/q"})

                # Add K Lemma
                if k_lemma.findtext("Text"):
                    lem_k = etree.SubElement(kq_rdgGrp, f"{{{tei_ns}}}lem", {"type": "k"})
                    w_k = etree.SubElement(lem_k, f"{{{tei_ns}}}w")
                    w_k.text = k_lemma.findtext("Text")

                # Add Q Lemma
                if q_lemma.findtext("Text"):
                    lem_q = etree.SubElement(kq_rdgGrp, f"{{{tei_ns}}}lem", {"type": "q"})
                    w_q = etree.SubElement(lem_q, f"{{{tei_ns}}}w")
                    w_q.text = q_lemma.findtext("Text")
            
            
            
            else:
                # Create a standard <lem> element if there's no range
                lem = etree.SubElement(app, f"{{{tei_ns}}}lem")

                # Process direct <Lemma> elements (when there's no From/To)
                direct_lemmas = lemmas_section.findall("Lemma")
                for lemma in direct_lemmas:
                    lemma_text = lemma.findtext("Text")
                    lemma_number = lemma.findtext("Number")

                    if lemma_text:
                        w = etree.SubElement(lem, f"{{{tei_ns}}}w")
                        w.text = lemma_text
                        if lemma_number:
                            w.set("n", lemma_number)  # Add @n if a number exists


        for decoded_entry in entry.findall("DecodedEntry"):
            #initialize some parameters
            witnesses = decoded_entry.findall("Details/Witnesses/Witness")
            reading_text = decoded_entry.findtext("Details/Rdg/Reading")
            sigla = decoded_entry.findtext("Details/Rdg/Sigla")  # Extract Sigla if available
          # Determine type: If sigla is ">", it represents an omission, etc.
            if sigla == ">":
                rdg_type = "omission"
            elif sigla == "+":
                rdg_type = "addition"
            else:
                rdg_type = sigla if sigla else "variant"
            
            # Create a reading group for each DecodedEntry
            rdgGrp = etree.SubElement(app, f"{{{tei_ns}}}rdgGrp", {"type": rdg_type})
            
            for witness in witnesses:
                manuscript = witness.findtext("Manuscript")
                collection = witness.findtext("Collection")

                if manuscript:
                    witness_id = f"wit-{manuscript}"  # Generate xml:id
                    full_name = manuscript if not collection else f"{collection} {manuscript}"  # Full name

                    # Store in witness mapping if not already added
                    if witness_id not in wit_mapping:
                        wit_entry = etree.SubElement(listWit, f"{{{tei_ns}}}witness", {f"{{{xml_ns}}}id": witness_id})
                        wit_entry.text = full_name
                        wit_mapping[witness_id] = full_name  # Save mapping

                else:
                    witness_id = "unknown_wit"  # Fallback for missing manuscript
                
                

                # Create <rdg> element
                rdg = etree.SubElement(rdgGrp, f"{{{tei_ns}}}rdg", {"wit": f"#{witness_id}", "type": rdg_type})

                # If there is a reading, add it inside <seg>
                if reading_text:
                    seg = etree.SubElement(rdg, f"{{{tei_ns}}}seg")
                    seg.text = reading_text
                    # Check for special sigla inside the reading
                    for siglum, explanation in special_sigla.items():
                        if siglum in reading_text:
                            sigla_id = sigla_id_mapping.get(siglum, f"sig-{re.sub(r'[^a-zA-Z0-9]+', '', siglum.strip())}")
                            # Store siglum definition if not already added
                            if sigla_id not in sigla_mapping:
                                prefix_def = etree.SubElement(listPrefixDef, f"{{{tei_ns}}}prefixDef", {f"{{{xml_ns}}}id": sigla_id})
                                desc = etree.SubElement(prefix_def, f"{{{tei_ns}}}desc")
                                desc.text = explanation
                                sigla_mapping[sigla_id] = explanation

                            # Reference the siglum in the seg element
                            seg.set("ref", f"#{sigla_id}")
                            seg.text = seg.text.replace(siglum, "").strip()
                            
                # If a witness comment exists, add it as a <note type="witness-comment">
                witness_comment = witness.findtext("Comment") if witness.findtext("Comment") else ""

                if witness_comment:
                    note = etree.SubElement(rdg, f"{{{tei_ns}}}note", {"type": "witness-comment"})
                    note.text = witness_comment
                
            # Extract general notes if any
            general_note = decoded_entry.findtext("Details/Rdg/GeneralComment")
            if general_note:
                note = etree.SubElement(rdgGrp, f"{{{tei_ns}}}note", {"type": "general"})
                note.text = general_note
                # Check for special sigla and create mapping if needed
                for siglum, explanation in special_sigla.items():
                    if siglum in general_note:
                        sigla_id = sigla_id_mapping.get(siglum, f"sig-{re.sub(r'[^a-zA-Z0-9]+', '', siglum.strip())}")

                        # Store siglum definition if not already added
                        if sigla_id not in sigla_mapping:
                            prefix_def = etree.SubElement(listPrefixDef, f"{{{tei_ns}}}prefixDef", {f"{{{xml_ns}}}id": sigla_id})
                            desc = etree.SubElement(prefix_def, f"{{{tei_ns}}}desc")
                            desc.text = explanation
                            sigla_mapping[sigla_id] = explanation

                        # Reference the siglum in the note
                        note.set("ref", f"#{sigla_id}")
                        note.text = note.text.replace(siglum, "").strip()
                
        # Handle cross-references and convert appid in Roman numerals
        cross_references = entry.findall("DecodedEntry/Details/CrossReferences/Reference")
        if cross_references:
            listRef = etree.SubElement(app, f"{{{tei_ns}}}listRef")
            for ref in cross_references:
                ref_text = ref.text.strip()
                if ref_text in roman_numerals:
                    ref_appid = roman_numerals[ref_text]  # Convert Roman numeral to digit
                    ref_el = etree.SubElement(
                        listRef, f"{{{tei_ns}}}ref",
                        {"target": f"app{ref_appid}-{chapter}-{verse}-{entry_number}"}
                    )
                    ref_el.text = ref_text  # Keep the original Roman numeral in text

    # Add import note if needed
        import_note = entry.findtext("Entry")
        if import_note:
            note_import = etree.SubElement(app, f"{{{tei_ns}}}note", {"type": "import"})
            note_import.text = import_note

    # Save the TEI XML
    tree = etree.ElementTree(tei)
    tree.write(output_file, pretty_print=True, xml_declaration=True, encoding="UTF-8")

# Run the conversion with appid = 3
input_xml = "Apparatus III Encoding 2.xml"
output_tei = "output_tei.xml"
convert_custom_xml_to_tei(input_xml, output_tei, appid=3)


In [ ]:
#new trials

In [22]:
from lxml import etree
import re

def create_tei_structure():
    """ Creates the TEI root and essential structure. """
    tei_ns = "http://www.tei-c.org/ns/1.0"
    xml_ns = "http://www.w3.org/XML/1998/namespace"

    tei = etree.Element(f"{{{tei_ns}}}TEI", nsmap={None: tei_ns, "xml": xml_ns})
    teiHeader = etree.SubElement(tei, f"{{{tei_ns}}}teiHeader")
    fileDesc = etree.SubElement(teiHeader, f"{{{tei_ns}}}fileDesc")
    titleStmt = etree.SubElement(fileDesc, f"{{{tei_ns}}}titleStmt")
    etree.SubElement(titleStmt, f"{{{tei_ns}}}title").text = "Converted TEI Document"
    pubStmt = etree.SubElement(fileDesc, f"{{{tei_ns}}}publicationStmt")
    pubStmt.text = "Generated from custom XML"
    sourceDesc = etree.SubElement(fileDesc, f"{{{tei_ns}}}sourceDesc")
    sourceDesc.text = "Converted from custom format"

    text = etree.SubElement(tei, f"{{{tei_ns}}}text")
    body = etree.SubElement(text, f"{{{tei_ns}}}body")
    listApp = etree.SubElement(body, f"{{{tei_ns}}}listApp")
    listWit = etree.SubElement(tei, f"{{{tei_ns}}}listWit")
    listPrefixDef = etree.SubElement(tei, f"{{{tei_ns}}}listPrefixDef")

    return tei, listApp, listWit, listPrefixDef, tei_ns, xml_ns


def process_entry(entry, listApp, listWit, listPrefixDef, tei_ns, xml_ns, appid):
    """ Processes an individual <entry> and adds it to <listApp>. """
    entry_number = entry.get("Number")
    chapter = entry.findtext("LemmaInfo/Chapter") or "unknown_chap"
    verse_from = entry.findtext("LemmaInfo/Verses/From")
    verse_to = entry.findtext("LemmaInfo/Verses/To")
    verse = verse_from if verse_from else entry.findtext("LemmaInfo/Verses")

    # ✅ Update `xml:id` to reflect ranges
    if verse_from and verse_to:
        app_id = f"app{appid}-{chapter}-{verse_from}-{verse_to}-{entry_number}"
    else:
        app_id = f"app{appid}-{chapter}-{verse}-{entry_number}"

    # ✅ Include `from` and `to` attributes if they exist
    app_attributes = {f"{{{xml_ns}}}id": app_id}
    if verse_from:
        app_attributes["from"] = verse_from
    if verse_to:
        app_attributes["to"] = verse_to

    app = etree.SubElement(listApp, f"{{{tei_ns}}}app", app_attributes)

    add_lemmas(entry, app, tei_ns, chapter, verse)
    handle_kq_lemmas(entry, app, tei_ns, chapter, verse)
    add_readings(entry, app, listWit, listPrefixDef, tei_ns, xml_ns)
    add_cross_references(entry, app, tei_ns, chapter, verse, entry_number)
    add_import_note(entry, app, tei_ns)


def add_lemmas(entry, app, tei_ns, chapter, verse):
    """ Processes lemmas, including handling ranges, while preventing duplication with k/q. """
    lemmas_section = entry.find("LemmaInfo/Lemmas")

    if lemmas_section is not None:
        has_kq = lemmas_section.find("Lemma[K='True']") is not None and lemmas_section.find("Lemma[Q='True']") is not None
        from_lemmas = lemmas_section.findall("From/Lemma")
        to_lemmas = lemmas_section.findall("To/Lemma")

        if from_lemmas and to_lemmas:
            lemGrp = etree.SubElement(app, f"{{{tei_ns}}}lemGrp", {"type": "range"})

            from_lemGrp = etree.SubElement(lemGrp, f"{{{tei_ns}}}lemGrp", {"type": "from"})
            from_lem = etree.SubElement(from_lemGrp, f"{{{tei_ns}}}lem")

            for lemma in from_lemmas:
                process_w_elements(from_lem, lemma, tei_ns, chapter, verse)

            to_lemGrp = etree.SubElement(lemGrp, f"{{{tei_ns}}}lemGrp", {"type": "to"})
            to_lem = etree.SubElement(to_lemGrp, f"{{{tei_ns}}}lem")

            for lemma in to_lemmas:
                process_w_elements(to_lem, lemma, tei_ns, chapter, verse)

        elif not has_kq:
            lem = etree.SubElement(app, f"{{{tei_ns}}}lem")
            for lemma in lemmas_section.findall("Lemma"):
                process_w_elements(lem, lemma, tei_ns, chapter, verse)  # ✅ Pass chapter & verse from entry level


def process_w_elements(lem_element, lemma, tei_ns, chapter, verse):
    """Processes <w> elements, ensuring every word has a @corresp attribute using entry-level chapter & verse."""
    lemma_text = lemma.findtext("Text")
    lemma_numbers = lemma.findtext("Number")

    if lemma_text:
        if lemma_numbers:
            numbers = lemma_numbers.split(",")  # ✅ Handle multiple numbers
            for num in numbers:
                num = num.strip()
                w = etree.SubElement(lem_element, f"{{{tei_ns}}}w")
                w.text = lemma_text
                w.set("n", num)  # ✅ Assign the lemma number
                
                # ✅ Generate @corresp using chapter & verse from entry level
                w.set("corresp", f"chapter{chapter}_verse{verse}_word{num}")
        else:
            w = etree.SubElement(lem_element, f"{{{tei_ns}}}w")
            w.text = lemma_text
            
            # ✅ Assign corresp even when there's no number
            w.set("corresp", f"chapter{chapter}_verse{verse}_wordX")  # Placeholder if no specific word number

                

def handle_kq_lemmas(entry, app, tei_ns, chapter, verse):
    """Processes Kethiv (k) and Qere (q) lemmas into a single <lem> with <choice> inside."""
    lemmas_section = entry.find("LemmaInfo/Lemmas")
    if lemmas_section is not None:
        k_lemma = lemmas_section.find("Lemma[K='True']")
        q_lemma = lemmas_section.find("Lemma[Q='True']")

        if k_lemma is not None and q_lemma is not None:
            # ✅ Create a single <lem> element (instead of separate k/q <lem>)
            lem = etree.SubElement(app, f"{{{tei_ns}}}lem")

            # ✅ Kethiv (<w> inside <lem>)
            k_text = k_lemma.findtext("Text")
            k_number = k_lemma.findtext("Number")

            if k_text:
                w_k = etree.SubElement(lem, f"{{{tei_ns}}}w")
                w_k.text = k_text
                if k_number:
                    w_k.set("n", k_number)  # ✅ Preserve lemma number
                w_k.set("corresp", f"chapter{chapter}_verse{verse}_wordID")  # ✅ Assign corresp

            # ✅ Qere (inside <choice>)
            q_text = q_lemma.findtext("Text")
            if q_text:
                choice = etree.SubElement(lem, f"{{{tei_ns}}}choice")  # ✅ Encapsulate Qere inside <choice>
                q_w = etree.SubElement(choice, f"{{{tei_ns}}}reg")  # ✅ Use <reg> for Qere
                q_w.text = q_text



def add_readings(entry, app, listWit, listPrefixDef, tei_ns, xml_ns):
    """ Processes readings into nested <rdgGrp> structures. """

    higher_rdgGrp = None  # Stores the upper-level related rdgGrp if needed
    last_variant_rdgGrp = None  # Stores the last <rdgGrp type="variant"> to attach related variants

    for decoded_entry in entry.findall("DecodedEntry"):
        witnesses = decoded_entry.findall("Details/Witnesses/Witness")
        reading_text = decoded_entry.findtext("Details/Rdg/Reading")
        sigla = decoded_entry.findtext("Details/Rdg/Sigla")
        general_note_text = decoded_entry.findtext("Details/Rdg/GeneralComment")

        entry_type = decoded_entry.findtext("Type")

        # ✅ Determine the lower rdgGrp type (addition, omission, modification)
        lower_rdgGrp_type = map_sigla(sigla) if entry_type != "Related Variant" else "modification"

        if entry_type == "Related Variant" and last_variant_rdgGrp is not None:
            # ✅ If a related variant appears, ensure a **higher rdgGrp** exists
            if higher_rdgGrp is None:
                higher_rdgGrp = etree.Element(f"{{{tei_ns}}}rdgGrp", {"type": "related"})
                app.append(higher_rdgGrp)  # ✅ Add higher rdgGrp to the main app
                higher_rdgGrp.append(last_variant_rdgGrp)  # ✅ Move existing variant inside related

            # ✅ Create a new lower rdgGrp inside the higher related rdgGrp
            target_rdgGrp = etree.SubElement(higher_rdgGrp, f"{{{tei_ns}}}rdgGrp", {"type": lower_rdgGrp_type})
        else:
            # ✅ This is a new, standalone variant
            rdgGrp = etree.SubElement(app, f"{{{tei_ns}}}rdgGrp", {"type": lower_rdgGrp_type})
            last_variant_rdgGrp = rdgGrp  # ✅ Store reference for possible related variants
            higher_rdgGrp = None  # ✅ Reset related group since we have a new independent variant
            target_rdgGrp = rdgGrp

        # ✅ Create <rdg> for the reading
        if reading_text:
            rdg = etree.SubElement(target_rdgGrp, f"{{{tei_ns}}}rdg")
            seg = etree.SubElement(rdg, f"{{{tei_ns}}}seg")
            clean_text, sigla_ref = process_sigla_in_text(reading_text.strip(), listPrefixDef, tei_ns, xml_ns)
            seg.text = clean_text
            if sigla_ref:
                seg.set("ref", sigla_ref)

        # ✅ Add witnesses (inside <rdgGrp>)
        for witness in witnesses:
            witness_id = process_witness(witness, listWit, tei_ns, xml_ns)
            witness_comment = witness.findtext("Comment")

            # ✅ Include witness comment in @ana attribute if it exists
            wit_attributes = {"wit": witness_id}
            if witness_comment:
                wit_attributes["ana"] = witness_comment  # ✅ Store witness comment inside `ana`

            wit = etree.SubElement(target_rdgGrp, f"{{{tei_ns}}}wit", wit_attributes)

        # ✅ Add general notes (inside <rdgGrp>)
        if general_note_text:
            note = etree.SubElement(target_rdgGrp, f"{{{tei_ns}}}note", {"type": "general"})
            cleaned_note, sigla_ref = process_sigla_in_text(general_note_text, listPrefixDef, tei_ns, xml_ns)
            note.text = cleaned_note
            if sigla_ref:
                note.set("ref", sigla_ref)


def process_witness(witness, listWit, tei_ns, xml_ns):
    """ Processes witness ID and adds it to <listWit> if not already present. """
    manuscript = witness.findtext("Manuscript")
    collection = witness.findtext("Collection")

    if manuscript:
        witness_id = f"wit-{manuscript}"
        full_name = manuscript if not collection else f"{collection} {manuscript}"

        # Define namespace mapping for XPath queries
        nsmap = {"tei": tei_ns, "xml": xml_ns}

        # Check if witness already exists before adding
        if not listWit.xpath("//tei:witness[@xml:id=$id]", namespaces=nsmap, id=witness_id):
            etree.SubElement(listWit, f"{{{tei_ns}}}witness", {f"{{{xml_ns}}}id": witness_id}).text = full_name

        return f"#{witness_id}"

    return "unknown_wit"



def map_sigla(sigla):
    """ Maps sigla symbols to TEI-compatible types. """
    sigla_mapping = {
        ">": "omission",
        "+": "addition"
    }
    return sigla_mapping.get(sigla, "variant")


def process_sigla_in_text(text, listPrefixDef, tei_ns, xml_ns):
    """ Detects, removes, and links sigla within readings & notes. """
    special_sigla = {
        "(!)": "Uncertain reading",
        "..": "Uncertain letters"
    }

    sigla_id_mapping = {
        "(!)": "sig-unc",
        "..": "sig-ell"
    }

    nsmap = {"tei": tei_ns, "xml": xml_ns}
    sigla_refs = []

    for siglum, desc in special_sigla.items():
        if siglum in text:
            sigla_id = sigla_id_mapping.get(siglum, f"sig-{re.sub(r'[^a-zA-Z0-9]+', '', siglum)}")

            # ✅ Ensure prefixDef exists (always add if missing)
            if not listPrefixDef.xpath(f"//tei:prefixDef[@xml:id='{sigla_id}']", namespaces=nsmap):
                prefix_def = etree.SubElement(listPrefixDef, f"{{{tei_ns}}}prefixDef", {f"{{{xml_ns}}}id": sigla_id})
                
                # ✅ Store the actual siglum marking
                prefix = etree.SubElement(prefix_def, f"{{{tei_ns}}}prefix")
                prefix.text = siglum  # ✅ Stores the siglum itself

                # ✅ Store the explanation
                desc_el = etree.SubElement(prefix_def, f"{{{tei_ns}}}desc")
                desc_el.text = desc  

            # ✅ Store the siglum reference
            sigla_refs.append(f"#{sigla_id}")

            # ✅ Remove siglum from text
            text = text.replace(siglum, "").strip()

    # ✅ Return cleaned text and **ALL** sigla references (if any)
    return text, " ".join(sigla_refs) if sigla_refs else None


def add_cross_references(entry, app, tei_ns, chapter, verse, entry_number):
    """ Converts Roman numeral references and adds them as <ref>. """
    roman_numerals = {
        "I": "1", "II": "2", "III": "3", "IV": "4", "V": "5", "VI": "6", "VII": "7", "VIII": "8", "IX": "9", "X": "10"
    }

    cross_references = entry.findall("DecodedEntry/Details/CrossReferences/Reference")
    if cross_references:
        listRef = etree.SubElement(app, f"{{{tei_ns}}}listRef")
        for ref in cross_references:
            ref_text = ref.text.strip()
            ref_appid = roman_numerals.get(ref_text, ref_text)  # Convert Roman numeral to number

            # ✅ Properly format the reference ID with converted numeral
            ref_el = etree.SubElement(listRef, f"{{{tei_ns}}}ref",
                                      {"target": f"app{ref_appid}-{chapter}-{verse}-{entry_number}"})
            ref_el.text = ref_text  # Keep the original Roman numeral in text

def add_import_note(entry, app, tei_ns):
    """ Adds an import note to the <app> element if available. """
    import_note = entry.findtext("Entry")
    if import_note:
        note_import = etree.SubElement(app, f"{{{tei_ns}}}note", {"type": "import"})
        note_import.text = import_note
            

def convert_custom_xml_to_tei(input_file, output_file, appid=3):
    tree = etree.parse(input_file)
    root = tree.getroot()
    tei, listApp, listWit, listPrefixDef, tei_ns, xml_ns = create_tei_structure()

    for entry in root.findall("entry"):
        process_entry(entry, listApp, listWit, listPrefixDef, tei_ns, xml_ns, appid)

    etree.ElementTree(tei).write(output_file, pretty_print=True, xml_declaration=True, encoding="UTF-8")


In [23]:
# Run the conversion with appid = 3
input_xml = "Apparatus III Encoding 2.xml"
output_tei = "output_tei.xml"
convert_custom_xml_to_tei(input_xml, output_tei, appid=3)


In [90]:
from lxml import etree
import re
import unicodedata
import re

# Define namespaces
TEI_NS = {"tei": "http://www.tei-c.org/ns/1.0"}

def strip_diacritics(text):
    """Removes Hebrew diacritics (nikkud & trop) from a given string."""
    if not text:
        return text  # Return None or empty string unchanged
    return "".join(c for c in unicodedata.normalize("NFD", text) if not re.match(r"[\u0590-\u05C7]", c))

def parse_base_text(base_text_file):
    """ Parses the base text and extracts word IDs mapped by text, chapter, and verse. """
    print("🔍 Parsing Base Text File...")

    tree = etree.parse(base_text_file)
    root = tree.getroot()

    word_map = {}
    total_words = 0  # Track total words processed

    # Find all verse elements
    verses = root.findall(".//tei:p[@type='verse']", namespaces=TEI_NS)
    print(f"📌 Found {len(verses)} <p type='verse'> elements.")

    for verse in verses:
        verse_id = verse.get("id")  # e.g., chapter1_verse1
        words = verse.findall(".//tei:w", namespaces=TEI_NS)

        if not words:
            print(f"⚠️ No words found in {verse_id}!")

        word_list = []  # Store words for sorting
        print(f"\n🔹 Processing Verse: {verse_id} → Found {len(words)} words")  # Debug print

        for word in words:
            word_id = word.get("id")  # e.g., chapter1_verse1_word1
            stripped_text = word.findtext("tei:stripped", namespaces=TEI_NS)

            if not word_id:
                print(f"⚠️ Missing ID for word: {etree.tostring(word, pretty_print=True).decode()}")

            if not stripped_text:
                print(f"⚠️ Missing stripped text for word ID {word_id}")

            if stripped_text and word_id:
                print(f"✅ Storing: {stripped_text} → {word_id}")  # Debug print
                word_list.append((stripped_text, word_id))
                total_words += 1  # Count each processed word

        # ✅ Store words sorted by word number (to ensure correct sequence)
        def extract_word_number(word_id):
            """ Extracts the numerical part from a word ID safely. """
            try:
                return int(re.search(r"_word(\d+)", word_id).group(1))
            except AttributeError:
                print(f"⚠️ Invalid word ID format: {word_id}")  # Debug print
                return float('inf')  # Place invalid values at the end

        word_map[verse_id] = sorted(word_list, key=lambda x: extract_word_number(x[1]))

    # ✅ Extract first 5 words across all verses
    all_words = [word for words in word_map.values() for word in words]  # Flatten to a list of words
#     print(f"First 5 words stored: {all_words[:5]}")
    print(f"✅ Base text parsing complete. Found {total_words} words in {len(word_map)} verses.")

    return word_map



def find_parent_app(element):
    """ Traverses up the XML tree to find the nearest <app> element. """
    while element is not None:
#         print(f"📌 First 5 words stored in word_map: {list(word_map.items())[:5]}")
        if element.tag == f"{{{TEI_NS['tei']}}}app":
            return element
        element = element.getparent()
    return None


def update_tei_with_corresp(tei_file, word_map, output_file, xml_ns):
    """ Updates the TEI file with correct word corresp values from the base text. """
    print(f"🔹 Loading TEI file: {tei_file}")

    tree = etree.parse(tei_file)
    root = tree.getroot()

    words_updated = 0
    total_words = 0

    # Track assigned words per verse to ensure sequential matching
    assigned_word_ids = {}

    # Find all <w> elements inside <lem>
    words = root.findall(".//tei:lem/tei:w", namespaces=TEI_NS)
    total_words = len(words)

    for word in words:
        word_text = strip_diacritics(word.text.strip()) if word.text else None
        lemma_parent = word.getparent()

        # Extract chapter & verse from the TEI structure
        app_element = lemma_parent.getparent()  # <app>
        #print(get_namespace(app_element))
        # Ensure the parent <app> is found
#         if app_element is None:
#             print(f"⚠️ No <app> found for word '{word_text}'. Skipping...")
#             continue
#         else:
#             print(f"✅ Found <app> for '{word_text}': {etree.tostring(app_element, pretty_print=True).decode()}")

        

        # Extract xml:id using the correct namespace
        app_id = app_element.get(f"{{http://www.w3.org/XML/1998/namespace}}id")

        if app_id:
            print(f"✅ Extracted app_id: {app_id} for word: {word_text}")
        if not app_id:
            print(f"⚠️ Missing app ID for word '{word_text}'. Skipping...")
            continue

        # Extract chapter & verse from xml:id
        chapter, verse = extract_chapter_verse(app_element)

        if not chapter or not verse:
            print(f"⚠️ Could not extract chapter/verse for {app_id}. Skipping...")
            continue

        print(f"🔍 Processing app: {app_id} → Chapter {chapter}, Verse {verse}")

        # Construct base text reference key
        verse_key = f"chapter{chapter}_verse{verse}"

#         print(f"🗂 word_map keys: {list(word_map.keys())[:10]}")  # Print first 10 keys
        
        # ✅ Extract first 5 words across all verses
        all_words = [word for words in word_map.values() for word in words]  # Flatten to a list of words
#         print(f"First 5 words stored: {all_words[:5]}")
        
        # 🔍 Print all available keys in word_map to check for mismatches
        if verse_key in word_map:
            available_words = [word[0] for word in word_map.get(verse_key, [])]
#             print(f"📌 Available words in {verse_key}: {available_words}")
# #          🔍 Show what is actually being searched vs. available options
#             print(f"🔍 Searching for: '{word_text}' in {verse_key}")
#             print(f"📌 Available words: {available_words}")
        if word_text not in available_words:
            print(f"⚠️ No match found for '{word_text}' in {verse_key}. Check word encoding and stripping.")

        
        number_attr = word.get("n")  # If word has "n" attribute

        # Initialize tracking for this verse
        if verse_key not in assigned_word_ids:
            assigned_word_ids[verse_key] = set()

        # Filter candidates in base text (store both word text & word ID)
        candidates = [(word[0], word[1]) for word in word_map.get(verse_key, [])]

        if candidates:
            if number_attr:
                numbers = number_attr.split(",")  # e.g., ["2", "3"]
                assigned_ids = []

                for num in numbers:
                    try:
                        idx = int(num) - 1  # Convert to index (1-based to 0-based)
                        if idx < len(candidates):
                            assigned_ids.append(candidates[idx][1])  # Assign the correct ID (word[1])
                        else:
                            print(f"⚠️ Index {idx} out of range for {word_text} in {verse_key}.")
                    except ValueError:
                        print(f"⚠️ Invalid number format: {num}")

                if assigned_ids:
                    word.set("corresp", " ".join(assigned_ids))
                    assigned_word_ids[verse_key].update(assigned_ids)
            else:
                # ✅ Find the next available word ID **after the last assigned word**
                last_assigned = max(assigned_word_ids[verse_key]) if assigned_word_ids[verse_key] else None

                for candidate_text, candidate_id in candidates:
                    # Ensure text matches and word hasn't been used
                    if candidate_text == word_text and candidate_id not in assigned_word_ids[verse_key]:  
                        # ✅ Ensure it appears **after** the last assigned word (if any)
                        word_number = int(candidate_id.split("_word")[-1])
                        if last_assigned:
                            last_number = int(last_assigned.split("_word")[-1])
                            if word_number <= last_number:
                                continue  # Skip words that come **before or at the same position**
#                         print(f"⚠️ BEFORE CHANGE: {word_text} → {word.get('corresp')}")
                        word.set("corresp", candidate_id)  
#                         print(f"✅ AFTER CHANGE: {word_text} → {word.get('corresp')}")
                        assigned_word_ids[verse_key].add(candidate_id)
                        break  # ✅ Ensure only the **next available word is assigned**

            words_updated += 1
            print(f"🔗 Updated {word_text} → {word.get('corresp')}")


        else:
            print(f"⚠️ No match found for '{word_text}' in {verse_key}")

    print(f"✅ TEI update complete: {words_updated}/{total_words} words updated.")

    # Save the updated TEI file
    tree.write(output_file, encoding="UTF-8", xml_declaration=True, pretty_print=True)
    print(f"📄 Updated TEI file saved as: {output_file}")


    
def extract_chapter_verse(app_element):
    """Extracts chapter and verse from an app xml:id."""
    # ✅ Correct namespace handling for xml:id
    
    app_id = app_element.get(f"{{http://www.w3.org/XML/1998/namespace}}id")
    # ✅ Extract numbers from app_id (e.g., app3-1-1-1 → chapter=1, verse=1)
    match = re.match(r"app\d+-(\d+)-(\d+)-\d+", app_id)
    if match:
        chapter, verse = match.groups()
        return chapter, verse  # ✅ Return as (chapter, verse)

#     print(f"⚠️ Could not extract chapter/verse from {app_id}.")
    return None, None


# File paths (update as needed)
base_text_file = "hosea_full_output_enhanced.xml"
tei_file = "output_tei.xml"
output_file = "final_output_tei.xml"

# Run the process
word_map = parse_base_text(base_text_file)
update_tei_with_corresp(tei_file, word_map, output_file, TEI_NS)


🔍 Parsing Base Text File...
📌 Found 205 <p type='verse'> elements.

🔹 Processing Verse: chapter1_verse1 → Found 22 words
✅ Storing: דבר → chapter1_verse1_word1
✅ Storing: יהוה → chapter1_verse1_word2
⚠️ Missing stripped text for word ID chapter1_verse1_word3
✅ Storing: אשר → chapter1_verse1_word4
✅ Storing: היה → chapter1_verse1_word5
✅ Storing: אל → chapter1_verse1_word6
✅ Storing: הושע → chapter1_verse1_word7
✅ Storing: בן → chapter1_verse1_word8
✅ Storing: בארי → chapter1_verse1_word9
✅ Storing: בימי → chapter1_verse1_word10
✅ Storing: עזיה → chapter1_verse1_word11
✅ Storing: יותם → chapter1_verse1_word12
✅ Storing: אחז → chapter1_verse1_word13
✅ Storing: יחזקיה → chapter1_verse1_word14
✅ Storing: מלכי → chapter1_verse1_word15
✅ Storing: יהודה → chapter1_verse1_word16
✅ Storing: ובימי → chapter1_verse1_word17
✅ Storing: ירבעם → chapter1_verse1_word18
✅ Storing: בן → chapter1_verse1_word19
✅ Storing: יואש → chapter1_verse1_word20
✅ Storing: מלך → chapter1_verse1_word21
✅ Storing: ישר

✅ Storing: לא → chapter8_verse5_word9
✅ Storing: יוכלו → chapter8_verse5_word10
✅ Storing: נקין → chapter8_verse5_word11

🔹 Processing Verse: chapter8_verse6 → Found 13 words
✅ Storing: כי → chapter8_verse6_word1
✅ Storing: מישראל → chapter8_verse6_word2
✅ Storing: והוא → chapter8_verse6_word3
✅ Storing: חרש → chapter8_verse6_word4
✅ Storing: עשהו → chapter8_verse6_word5
✅ Storing: ולא → chapter8_verse6_word6
✅ Storing: אלהים → chapter8_verse6_word7
✅ Storing: הוא → chapter8_verse6_word8
✅ Storing: כי → chapter8_verse6_word9
✅ Storing: שבבים → chapter8_verse6_word10
✅ Storing: יהיה → chapter8_verse6_word11
✅ Storing: עגל → chapter8_verse6_word12
✅ Storing: שמרון → chapter8_verse6_word13

🔹 Processing Verse: chapter8_verse7 → Found 16 words
✅ Storing: כי → chapter8_verse7_word1
✅ Storing: רוח → chapter8_verse7_word2
✅ Storing: יזרעו → chapter8_verse7_word3
✅ Storing: וסופתה → chapter8_verse7_word4
✅ Storing: יקצרו → chapter8_verse7_word5
✅ Storing: קמה → chapter8_verse7_word6
✅ Storing:

🔗 Updated את → chapter5_verse7_word10
✅ Extracted app_id: app3-5-8-100 for word: חצצרה
🔍 Processing app: app3-5-8-100 → Chapter 5, Verse 8
🔗 Updated חצצרה → chapter5_verse8_word5
✅ Extracted app_id: app3-5-8-101 for word: בית
🔍 Processing app: app3-5-8-101 → Chapter 5, Verse 8
🔗 Updated בית → chapter5_verse8_word8
✅ Extracted app_id: app3-5-8-102 for word: אחריך
🔍 Processing app: app3-5-8-102 → Chapter 5, Verse 8
🔗 Updated אחריך → chapter5_verse8_word10
✅ Extracted app_id: app3-5-10-103 for word: כמסיגי
🔍 Processing app: app3-5-10-103 → Chapter 5, Verse 10
🔗 Updated כמסיגי → chapter5_verse10_word4
✅ Extracted app_id: app3-5-10-104 for word: גבול
🔍 Processing app: app3-5-10-104 → Chapter 5, Verse 10
🔗 Updated גבול → chapter5_verse10_word5
✅ Extracted app_id: app3-5-11-105 for word: אחרי
🔍 Processing app: app3-5-11-105 → Chapter 5, Verse 11
🔗 Updated אחרי → chapter5_verse11_word8
✅ Extracted app_id: app3-5-12-106 for word: לאפרים
🔍 Processing app: app3-5-12-106 → Chapter 5, Verse 12
🔗 Up

In [ ]:
#old function
def add_readings(entry, app, listWit, listPrefixDef, tei_ns, xml_ns):
    """ Processes readings and creates <rdgGrp> and <rdg> elements. """
    
    for decoded_entry in entry.findall("DecodedEntry"):
        witnesses = decoded_entry.findall("Details/Witnesses/Witness")
        reading_text = decoded_entry.findtext("Details/Rdg/Reading")
        sigla = decoded_entry.findtext("Details/Rdg/Sigla")
        general_note_text = decoded_entry.findtext("Details/Rdg/GeneralComment")  # ✅ Capture General Notes

        rdg_type = map_sigla(sigla)
        rdgGrp = etree.SubElement(app, f"{{{tei_ns}}}rdgGrp", {"type": rdg_type})

        for witness in witnesses:
            witness_id = process_witness(witness, listWit, tei_ns, xml_ns)
            witness_comment = witness.findtext("Comment")

            rdg = etree.SubElement(rdgGrp, f"{{{tei_ns}}}rdg", {"wit": witness_id, "type": rdg_type})

            if reading_text:
                seg = etree.SubElement(rdg, f"{{{tei_ns}}}seg")
                clean_text, sigla_ref = process_sigla_in_text(reading_text.strip(), listPrefixDef, tei_ns, xml_ns)
                seg.text = clean_text
                if sigla_ref:
                    seg.set("ref", sigla_ref)  # ✅ Link to siglum definition

            # ✅ Add witness comment inside <rdg>
            if witness_comment:
                note = etree.SubElement(rdg, f"{{{tei_ns}}}note", {"type": "witness-comment"})
                note.text = witness_comment

        # ✅ Add general note inside <rdgGrp> if it exists
        if general_note_text:
            general_note = etree.SubElement(rdgGrp, f"{{{tei_ns}}}note", {"type": "general"})
            cleaned_note, sigla_ref = process_sigla_in_text(general_note_text, listPrefixDef, tei_ns, xml_ns)
            general_note.text = cleaned_note
            if sigla_ref:
                general_note.set("ref", sigla_ref)  # ✅ Link to siglum definition

In [ ]:
############## custom-xml parser 

In [7]:
def convert_doc_to_docx(doc_path):
    word = client.Dispatch("Word.Application")
    absolute_doc_path = os.path.abspath(doc_path)
    doc = word.Documents.Open(absolute_doc_path)
    doc_path_new = absolute_doc_path.replace(".doc", ".docx")
    doc.SaveAs2(doc_path_new, FileFormat=16)  # FileFormat=16 is for docx
    doc.Close()
    word.Quit()
    return doc_path_new

# Example Usage
docx_path = convert_doc_to_docx('Hosea.2.doc')

def convert_docx_to_txt(docx_file_path, txt_file_path):
    # Load the .docx file
    doc = Document(docx_file_path)

    # Extract text from each paragraph in the document
    text_content = '\n'.join([paragraph.text for paragraph in doc.paragraphs])

    # Write the extracted text to a .txt file
    with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text_content)

    print(f"File converted successfully and saved as '{txt_file_path}'")

# Example usage

convert_docx_to_txt('Hosea.2.docx', 'output3.txt')


File converted successfully and saved as 'output3.txt'


In [2]:
with open('01 Hosea App III - מתוקן.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    txt = file
lines

['\ufeffApp III: Hosea\n',
 '\n',
 'Chapter 1\n',
 '1 עזיה] 30 עזיהו\n',
 'יחזקיה] 30 93 (pm) 96 יחזקיהו\n',
 'ירבעם בן] 30 + נבט (non voc)\n',
 '2 בהושע] 93 (pm) ביהושעIV (similarly PesiqtaR 33 (153b))\n',
 '3 ותלד] 150 (pm) + בן\n',
 ' לו] 96 >I  II IV\n',
 '4 ופקדתי] 30 + על (non voc)\n',
 'ממלכוּת] 96 ממלכוֹת\n',
 '5 את] 30 >\n',
 'יזרעאל] 150 ישראל (parall; but 150-Tg: יזרעאל)\n',
 '6 עוד1] 30 (pm) >I II IV (similarly b. Pesaḥim 87bmss)\n',
 'כי] 93 (pm) + את\n',
 'אוסיף] 150 (pm) >\n',
 'את] 93 (pm) >\n',
 '7 והושעתים] 93 (pm) והושעתם\n',
 'בסוסים] 96 ובסוסיםI IV\n',
 '8 רֻחמה] 30 לח..\n',
 '9 ואנכי] 150 (pm) אנכי | 93 ואני\n',
 '\n',
 'Chapter 2\n',
 '1 אשר1] 150 + יאמר (non voc)\n',
 'אשר2] 96 + לא\n',
 '2 להם] 30 (pm) לכם\n',
 'אחד] 30 (pm) 150 (pm) >\n',
 '4 ונאפופיה] 150 (pm) ונאפו\n',
 '5 ערֻמה] 96 ערוֹמה \n',
 'והצגתיה] 93 (pm) + כיום ערומה והצגתיה \n',
 'ושתִּה] 150 (pm) ושמתיה\n',
 '7 אלכה] 30 + ואשיבה אל איש (non voc)\n',
 '8 שָׂך] 30 (pm) 93 (pm) 150 (pm) סךII IV (See 

In [ ]:
########## render apparatus 3 ##############

In [ ]:
#new logic
#1. split into lemma and app_entry. process each separately
# lemma: split into number range and lemma range. 
#numbers include \d+'-', lemmas need to include potential k\q attribute, and word number.
# app_entry: splits into mss., reading, comments, cross-reference.
# 


In [15]:
#processing functions, including chapter information
def process_full_entry(text, chapter, previous_verse=None):
    lemma, part_entry = split_full_entry(text)
    lemma_dict = lemma_verse_processor(lemma, chapter)
    
    # Use the previous verse if the current verse list is empty
    if not lemma_dict['verses'] and previous_verse is not None:
        lemma_dict['verses'] = previous_verse
        

    # Split part_entry by '|'
    if '|' in part_entry:
        entry_parts = part_entry.split('|')
    else:
        entry_parts = [part_entry]

    # Initialize a list to hold all processed parts
    processed_parts = []

    # Process each part separately
    for part in entry_parts:
        # Split part by ',' not inside parentheses
        sub_parts = split_on_comma_not_in_parentheses(part)

        # Process each sub-part using process_comma_entry
        processed_sub_parts = [process_comma_entry(sub_part) for sub_part in sub_parts]

        # Concatenate processed sub-parts for each part
        processed_parts.append(processed_sub_parts)

    # Combine processed parts. Assuming you want them as a nested list
    decoded_entries = processed_parts

    # Return the lemma_dict and decoded_entries, along with the verses used for this entry
    return lemma_dict, decoded_entries, lemma_dict['verses']

def split_on_comma_not_in_parentheses(part):
    """
    Splits the string on ',' not inside parentheses.
    """
    sub_parts = []
    current_part = []
    paren_depth = 0  # Track depth of parentheses

    for char in part:
        if char == '(':
            paren_depth += 1
        elif char == ')':
            paren_depth -= 1
        elif char == ',' and paren_depth == 0:
            # At a top-level comma, split here
            sub_parts.append(''.join(current_part))
            current_part = []
            continue

        current_part.append(char)

    # Add the last part if there's any
    if current_part:
        sub_parts.append(''.join(current_part))

    return sub_parts

def split_full_entry(text):
    sliced_entry = text.split(sep=']')
    lemma, entry = sliced_entry
#         print(f"lemma: {lemma}")
#         print(f"entry: {entry}")
    return lemma, entry    

# def lemma_verse_processor(text, chapter):
#     # Simplified approach: first split into digits and lemmas
#     # Regex to match the verse numbers at the beginning
#     verse_regex = r'^(\d+(?:–\d+)?)\s'
    
#     # Extract verses
#     verses_match = re.match(verse_regex, text)
#     verses = list(map(int, verses_match.group(1).split('–'))) if verses_match else []
    
#     # Isolate lemmas part by removing the verses
#     lemmas_part = text[len(verses_match.group(0)):].strip() if verses_match else text
#     return {
#         'chapter': chapter,
#         'verses': verses,
#         'lemmas': process_lemma_with_range_and_diacritics(lemmas_part)
#     }

def lemma_verse_processor(text, chapter):
    # Regex to match the verse numbers at the beginning
    verse_regex = r'^(\d+(?:–\d+)?)\s'
    
    # Extract verses
    verses_match = re.match(verse_regex, text)
    if verses_match:
        verse_range = verses_match.group(1).split('–')
        if len(verse_range) == 2 and verse_range[0] != verse_range[1]:
            verses = {'from': int(verse_range[0]), 'to': int(verse_range[1])}
        else:
            verses = int(verse_range[0])
    else:
        verses = None
    
    # Isolate lemmas part by removing the verses
    lemmas_part = text[len(verses_match.group(0)):].strip() if verses_match else text
    return {
        'chapter': chapter,
        'verses': verses,
        'lemmas': process_lemma_with_range_and_diacritics(lemmas_part)
    }


# Function to process individual lemmas or ranges, after the split,
lemma_regex = r'(k|q)?\s*([^\d\s]+)(\d?\,?\d?)'#(\d+(?:–\d+)?)\s

def process_lemma_with_range_and_diacritics(lemma):
    # Adjust regex to include diacritical marks and punctuation within Hebrew words
    
    
    # Check for range indicated by "–" and process accordingly
    if "–" in lemma:
        from_lemma, to_lemma = lemma.split("–")
        return {
            'from': process_individual_lemma(from_lemma.strip()),
            'to': process_individual_lemma(to_lemma.strip())
        }

    # Split lemma if there are separate lemmas with "/"
    split_lemmas = re.split(r'\s*/\s*', lemma) if '/' in lemma else [lemma]
    
    processed_lemmas = []
    for split_lemma in split_lemmas:
        processed = process_individual_lemma(split_lemma)
        processed_lemmas.extend(processed)
    
    return processed_lemmas

def process_individual_lemma(individual_lemma):
    matches = re.findall(lemma_regex, individual_lemma)
    processed_lemmas = []
    for match in matches:
        prefix, word, number = match
        lemma_dict = {'lemma': word}
        if prefix: lemma_dict[prefix] = True
        if number: lemma_dict['number'] = (number)
        processed_lemmas.append(lemma_dict)
    return processed_lemmas

# processing functions for sub-units of app_entry, for which there is matching lemma and verse data processed above

def extract_cross_references(text): #extract cross-references
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

def parse_witnesses(text):
    pattern = re.compile(r'\s?([^\d]*?)?(\d+)\s?\(?([^\)\d.]+)?\)?', re.DOTALL | re.UNICODE)
    parts = re.findall(pattern, text)
    # Filter out empty tuples
    return [part for part in parts if any(part)]

def post_process_witnesses(witnesses):
    # Define the set of specific values for "x"
    specific_values = {"G-B Eb ", "G-B Kb ", "G-B Msr "}  # Replace with the actual values

    # Iterate over the witnesses, except for the last one
    for i in range(len(witnesses) - 1):
        z, y, x = witnesses[i]

        # Check if "x" is one of the specific values
        if x in specific_values:
            # Remove "x" from the current tuple and prepend it to the "z" of the next witness
            witnesses[i] = (z, y, '')
            next_z, next_y, next_x = witnesses[i + 1]
            witnesses[i + 1] = (x + next_z, next_y, next_x)

    # Remove the first tuple if it becomes empty
    if witnesses and witnesses[0] == ('', '', ''):
        witnesses.pop(0)

    return witnesses

def parse_comma_witnesses(text):
    pattern = re.compile(r'\s?([^\d]*?)?(\d*)?\s?\(?([^\)\d]+)?\)?', re.DOTALL | re.UNICODE)
    parts = re.findall(pattern, text)
    # Filter out empty tuples
    return [part for part in parts if any(part)]

def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~\.]*)                         # Captures special sigla
        \s*
        (?P<Reading>(?:[kq]*\s?)[\u0590-\u05FF\uFB1D-\uFB4F\s.]*    # Hebrew reading, including 'k', 'q'
                   (?:/\s(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*)?)  # Allows for 'k'/'q' followed by Hebrew, separated by '/'
        \s*
        \s*
        (?P<GeneralComment>\(.*\))?                     # Captures comments
    """
    
    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v for k, v in match.groupdict().items() if v}

    return parsed_entry

#splitting entry into witnesses and reading (if only one group assign to witnesses)
def witness_reading_splitter(text):
    pattern = re.compile(r'(.*?)?([\+<>~.]*\s?[kq\u0590-\u05FF]+)(.*)?', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        pattern = re.compile(r'(.*?)([\+<>~])(.*)?', re.DOTALL)
        match = pattern.match(text)
        if match:
            return match.groups()  # Returns a tuple with the three parts
        else:
            return text
        return text  # No divider matching the pattern was found


def process_entry(entry):
    clean_entry, cross_references = extract_cross_references(entry)
    split_entry = witness_reading_splitter(clean_entry)
    if type(split_entry) is tuple:
        witnesses = {'Witnesses': parse_witnesses(split_entry[0])}
        if len(split_entry) == 2:
            reading = parse_reading_entry(split_entry[1])
        else:  # there are 3 groups:
            reading = parse_reading_entry(split_entry[1] + split_entry[2])
    else:
        witnesses = {'Witnesses': parse_witnesses(split_entry)}
        reading = ''
    # Include "Cross References" only if the list is not empty
    result = [witnesses, {"Rdg": reading}]
    if cross_references:
        result.append({"Cross References": cross_references})
    return result

def process_comma_entry(entry):
    clean_entry, cross_references = extract_cross_references(entry)
    split_entry = witness_reading_splitter(clean_entry)
    if type(split_entry) is tuple:
        witnesses = {'Witnesses': parse_comma_witnesses(split_entry[0])}
        if len(split_entry) == 2:
            reading = parse_reading_entry(split_entry[1])
        else:  # there are 3 groups:
            reading = parse_reading_entry(split_entry[1] + split_entry[2])
    else:
        witnesses = {'Witnesses': parse_comma_witnesses(split_entry)}
        reading = ''
    # Include "Cross References" only if the list is not empty
    result = [witnesses, {"Rdg": reading}]
    if cross_references:
        result.append({"Cross References": cross_references})
    return result


In [18]:
##### 
def compile_and_display_entries(entries):
    previous_verse = None
    current_chapter = None
    compiled_results = []
    
    for entry in entries:
        chapter_match = re.search(r'Chapter (\d+)', entry)
        if chapter_match:
            current_chapter = chapter_match.group(1)
            continue
        elif entry.strip() and not entry.strip().isdigit():
            clean_entry = entry.strip()
            lemma_dict, decoded_entries, used_verse = process_full_entry(clean_entry, current_chapter, previous_verse)
            previous_verse = used_verse

            # Combine the lemma_dict and decoded_entries for display
            entry_result = {
                #'Chapter': current_chapter,
                'Entry': clean_entry,
                'Lemma Info': lemma_dict,
                'Decoded Entries': []
            }
            last_witness_second_group = None
            for i, decoded_entries_list in enumerate(decoded_entries):
                for j, decoded_entry in enumerate(decoded_entries_list):
                    variant_type = "Variant" if j == 0 else "Related Variant"
                    variant_dict = {'Type': variant_type, 'Details': []}
                    for item in decoded_entry:
                        if isinstance(item, dict) and 'Witnesses' in item:
                            # Update witnesses with last non-empty second group if needed
                            updated_witnesses = []
                            for witness in item['Witnesses']:
                                if witness[1] == '' and last_witness_second_group:
                                    updated_witnesses.append((witness[0], last_witness_second_group, witness[2]))
                                else:
                                    updated_witnesses.append(witness)
                                    if witness[1]:
                                        last_witness_second_group = witness[1]
                            # Apply post-processing to witnesses
                            item['Witnesses'] = post_process_witnesses(updated_witnesses)
                        variant_dict['Details'].append(item)
                    entry_result['Decoded Entries'].append(variant_dict)
            compiled_results.append(entry_result)

    # Display the results in a structured format
    for result in compiled_results:
        print(f"Entry:")
        print(f"{result['Entry']}")
        print("Lemma Info:")
        for key, value in result['Lemma Info'].items():
            print(f"  {key}: {value}")
        print("Decoded Entries:")
        for variant in result['Decoded Entries']:
            print(f"  {variant['Type']}:")
            for detail in variant['Details']:
                if isinstance(detail, dict):
                    for key, value in detail.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {detail}")
        print("-" * 50)  # Separator line
    
    return compiled_results

# Example usage
test = compile_and_display_entries(lines[2:-7]) # notice
test

Entry:
1 עזיה] 30 עזיהו
Lemma Info:
  chapter: 1
  verses: 1
  lemmas: [{'lemma': 'עזיה'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', '')]
    Rdg: {'Reading': 'עזיהו'}
--------------------------------------------------
Entry:
יחזקיה] 30 93 (pm) 96 יחזקיהו
Lemma Info:
  chapter: 1
  verses: 1
  lemmas: [{'lemma': 'יחזקיה'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', ''), ('', '93', 'pm'), ('', '96', '')]
    Rdg: {'Reading': 'יחזקיהו'}
--------------------------------------------------
Entry:
ירבעם בן] 30 + נבט (non voc)
Lemma Info:
  chapter: 1
  verses: 1
  lemmas: [{'lemma': 'ירבעם'}, {'lemma': 'בן'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', '')]
    Rdg: {'Sigla': '+', 'Reading': 'נבט ', 'GeneralComment': '(non voc)'}
--------------------------------------------------
Entry:
2 בהושע] 93 (pm) ביהושעIV (similarly PesiqtaR 33 (153b))
Lemma Info:
  chapter: 1
  verses: 2
  lemmas: [{'lemma': 'בהושע'}]
Decoded Entries:
  Variant:
    Witnesses: [(''

  lemmas: [{'lemma': 'שרים'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '96', ''), ('', '150', '')]
    Rdg: {'Reading': 'ושרים  '}
    Cross References: ['I', 'II', 'IV']
--------------------------------------------------
Entry:
11 כי] 89 (pm) >
Lemma Info:
  chapter: 8
  verses: 11
  lemmas: [{'lemma': 'כי'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '89', 'pm')]
    Rdg: {'Sigla': '>'}
--------------------------------------------------
Entry:
אפרים] 30 (pm) ישראל
Lemma Info:
  chapter: 8
  verses: 11
  lemmas: [{'lemma': 'אפרים'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', 'pm')]
    Rdg: {'Reading': 'ישראל'}
--------------------------------------------------
Entry:
לו] 93 (pm) לי
Lemma Info:
  chapter: 8
  verses: 11
  lemmas: [{'lemma': 'לו'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '93', 'pm')]
    Rdg: {'Reading': 'לי'}
--------------------------------------------------
Entry:
12 k אכתוב / q אכתב] 93 96 150 (sm) k, 30 89 150 (pm) q
Lemma Info:


[{'Entry': '1 עזיה] 30 עזיהו',
  'Lemma Info': {'chapter': '1', 'verses': 1, 'lemmas': [{'lemma': 'עזיה'}]},
  'Decoded Entries': [{'Type': 'Variant',
    'Details': [{'Witnesses': [('', '30', '')]},
     {'Rdg': {'Reading': 'עזיהו'}}]}]},
 {'Entry': 'יחזקיה] 30 93 (pm) 96 יחזקיהו',
  'Lemma Info': {'chapter': '1', 'verses': 1, 'lemmas': [{'lemma': 'יחזקיה'}]},
  'Decoded Entries': [{'Type': 'Variant',
    'Details': [{'Witnesses': [('', '30', ''),
       ('', '93', 'pm'),
       ('', '96', '')]},
     {'Rdg': {'Reading': 'יחזקיהו'}}]}]},
 {'Entry': 'ירבעם בן] 30 + נבט (non voc)',
  'Lemma Info': {'chapter': '1',
   'verses': 1,
   'lemmas': [{'lemma': 'ירבעם'}, {'lemma': 'בן'}]},
  'Decoded Entries': [{'Type': 'Variant',
    'Details': [{'Witnesses': [('', '30', '')]},
     {'Rdg': {'Sigla': '+',
       'Reading': 'נבט ',
       'GeneralComment': '(non voc)'}}]}]},
 {'Entry': '2 בהושע] 93 (pm) ביהושעIV (similarly PesiqtaR 33 (153b))',
  'Lemma Info': {'chapter': '1', 'verses': 2, 'lem

In [19]:
def list_to_xml(data, output_file):
    root = ET.Element("root")

    for idx, entry in enumerate(data, start=1):
        entry_element = ET.SubElement(root, "entry", {"Number": str(idx)})
        ET.SubElement(entry_element, "Entry").text = entry.get("Entry", "")

        lemma_info = entry.get("Lemma Info", {})
        if isinstance(lemma_info, dict):
            lemma_info_element = ET.SubElement(entry_element, "LemmaInfo")
            chapter = lemma_info.get("chapter", "")
            if isinstance(chapter, str):
                ET.SubElement(lemma_info_element, "Chapter").text = chapter
            verses = lemma_info.get("verses", "")
            if isinstance(verses, (str, int)):
                ET.SubElement(lemma_info_element, "Verses").text = str(verses)
            elif isinstance(verses, dict):
                verses_element = ET.SubElement(lemma_info_element, "Verses")
                for key, value in verses.items():
                    ET.SubElement(verses_element, key.capitalize()).text = str(value)
            lemmas_element = ET.SubElement(lemma_info_element, "Lemmas")
            lemmas = lemma_info.get("lemmas", [])
            if isinstance(lemmas, list):
                for lemma in lemmas:
                    lemma_element = ET.SubElement(lemmas_element, "Lemma")
                    ET.SubElement(lemma_element, "Text").text = lemma.get("lemma", "")
                    if "number" in lemma:
                        ET.SubElement(lemma_element, "Number").text = str(lemma["number"])
                    if "k" in lemma:
                        ET.SubElement(lemma_element, "K").text = str(lemma["k"])
                    if "q" in lemma:
                        ET.SubElement(lemma_element, "Q").text = str(lemma["q"])
            elif isinstance(lemmas, dict):
                for key, lemma_list in lemmas.items():
                    lemma_range_element = ET.SubElement(lemmas_element, key.capitalize())
                    for lemma in lemma_list:
                        lemma_element = ET.SubElement(lemma_range_element, "Lemma")
                        ET.SubElement(lemma_element, "Text").text = lemma.get("lemma", "")
                        if "k" in lemma:
                            ET.SubElement(lemma_element, "K").text = str(lemma["k"])
                        if "q" in lemma:
                            ET.SubElement(lemma_element, "Q").text = str(lemma["q"])

        decoded_entries = entry.get("Decoded Entries", [])
        for decoded_entry in decoded_entries:
            if isinstance(decoded_entry, dict):
                decoded_entry_element = ET.SubElement(entry_element, "DecodedEntry")
                ET.SubElement(decoded_entry_element, "Type").text = decoded_entry.get("Type", "")
                details_element = ET.SubElement(decoded_entry_element, "Details")
                for detail in decoded_entry.get("Details", []):
                    if isinstance(detail, dict):
                        if "Witnesses" in detail:
                            witnesses_element = ET.SubElement(details_element, "Witnesses")
                            for witness in detail["Witnesses"]:
                                if isinstance(witness, tuple):
                                    witness_element = ET.SubElement(witnesses_element, "Witness")
                                    ET.SubElement(witness_element, "Collection").text = witness[0]
                                    ET.SubElement(witness_element, "Manuscript").text = witness[1]
                                    ET.SubElement(witness_element, "Comment").text = witness[2]
                        elif "Rdg" in detail:
                            rdg_element = ET.SubElement(details_element, "Rdg")
                            rdg = detail["Rdg"]
                            if isinstance(rdg, dict):
                                if "Sigla" in rdg:
                                    ET.SubElement(rdg_element, "Sigla").text = rdg["Sigla"]
                                ET.SubElement(rdg_element, "Reading").text = rdg.get("Reading", "")
                                if "GeneralComment" in rdg:
                                    ET.SubElement(rdg_element, "GeneralComment").text = rdg["GeneralComment"]
                        elif "Cross References" in detail:
                            cross_references_element = ET.SubElement(details_element, "CrossReferences")
                            for reference in detail["Cross References"]:
                                ET.SubElement(cross_references_element, "Reference").text = reference

    tree = ET.ElementTree(root)
    tree.write(output_file, encoding="utf-8", xml_declaration=True)

# Example usage
list_to_xml(test, "Apparatus III Encoding 2.xml")


In [171]:
def create_multiindex_df(compiled_results):
    # Prepare data for DataFrame
    data = []
    index = []
    for result in compiled_results:
        entry = result['Entry']
        lemma_info = result['Lemma Info']
        chapter = lemma_info['chapter']
        verses = lemma_info['verses']
        if isinstance(verses, dict):  # Check if verses have "from" and "to" keys
            verse_from = verses['from']
            verse_to = verses['to']
        else:
            verse_from = verse_to = verses

        lemmas_from = lemmas_to = []
        if isinstance(lemma_info['lemmas'], list):
            lemmas = [lemma['lemma'] for lemma in lemma_info['lemmas']]
        else:
            lemmas_from = [lemma['lemma'] for lemma in lemma_info['lemmas']['from']]
            lemmas_to = [lemma['lemma'] for lemma in lemma_info['lemmas']['to']]
            lemmas = lemmas_from + lemmas_to

        for variant in result['Decoded Entries']:
            variant_type = variant['Type']
            for lemma in lemmas:
                lemma_range = 'from' if lemma in lemmas_from else 'to' if lemma in lemmas_to else None
                index_key = (chapter, verse_from if lemma_range == 'from' else verse_to, lemma, variant_type)
                row = {'Entry': entry, 'Lemma Range': lemma_range}
                for detail in variant['Details']:
                    if isinstance(detail, dict):
                        if 'Rdg' in detail:
                            if isinstance(detail['Rdg'], dict):
                                rdg = detail['Rdg']
                                reading = rdg.get('Reading', None)
                                if reading == 'k':
                                    reading = f"k ({lemmas_from[0]})" if lemmas_from else 'k'
                                elif reading == 'q':
                                    reading = f"q ({lemma})" if lemma in lemmas_to else 'q'
                                row['Reading'] = reading
                                row['Sigla'] = rdg.get('Sigla', None)
                                row['Comment'] = rdg.get('Comment', None)
                            else:
                                row['Reading'] = detail['Rdg']
                        else:
                            row.update(detail)
                    else:
                        row['Detail'] = detail
                index.append(index_key)
                data.append(row)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(data)

    # Create MultiIndex for rows
    multi_index = pd.MultiIndex.from_tuples(index, names=['Chapter', 'Verse', 'Lemma', 'Variant Type'])

    # Assign MultiIndex to DataFrame
    df.index = multi_index

    return df

# Create MultiIndex DataFrame from compiled_results
compiled_results_df = create_multiindex_df(test)

# Display the DataFrame
compiled_results_df.head()


Entry  \
Chapter Verse Lemma  Variant Type                                                      
1       1     עזיה   Variant                                        1 עזיה] 30 עזיהו   
              יחזקיה Variant                           יחזקיה] 30 93 (pm) 96 יחזקיהו   
              ירבעם  Variant                            ירבעם בן] 30 + נבט (non voc)   
              בן     Variant                            ירבעם בן] 30 + נבט (non voc)   
        2     בהושע  Variant       2 בהושע] 93 (pm) ביהושעIV (similarly PesiqtaR ...   

                                  Lemma Range  \
Chapter Verse Lemma  Variant Type               
1       1     עזיה   Variant             None   
              יחזקיה Variant             None   
              ירבעם  Variant             None   
              בן     Variant             None   
        2     בהושע  Variant             None   

                                                          Witnesses  Reading  \
Chapter Verse Lemma  Variant Type                                              
1       1     עזיה   Variant                             [(, 30, )]    עזיהו   
              יחזקיה Variant       [(, 30, ), (, 93, pm), (, 96, )]  יחזקיהו   
              ירבעם  Variant                             [(, 30, )]     נבט    
              בן     Variant                             [(, 30, )]     נבט    
        2     בהושע  Variant                           [(, 93, pm)]  ביהושע    

                                  Sigla                         Comment  \
Chapter Verse Lemma  Variant Type                                         
1       1     עזיה   Variant       None                            None   
              יחזקיה Variant       None                            None   
              ירבעם  Variant          +                       (non voc)   
              בן     Variant          +                       (non voc)   
        2     בהושע  Variant       None  (similarly PesiqtaR 33 (153b))   

                                  Cross References  
Chapter Verse Lemma  Variant Type                   
1       1     עזיה   Variant                   NaN  
              יחזקיה Variant                   NaN  
              ירבעם  Variant                   NaN  
              בן     Variant                   NaN  
        2     בהושע  Variant                  [IV]

In [156]:
compiled_results_df.to_excel('Trial Master.xlsx')

<ipython-input-156-727d52b46ba7>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  compiled_results_df.to_excel('Trial Master.xlsx')


In [135]:
test

[{'Entry': '1 עזיה] 30 עזיהו',
  'Lemma Info': {'chapter': '1', 'verses': 1, 'lemmas': [{'lemma': 'עזיה'}]},
  'Decoded Entries': [{'Type': 'Variant',
    'Details': [{'Witnesses': [('', '30', '')]},
     {'Rdg': {'Reading': 'עזיהו'}}]}]},
 {'Entry': 'יחזקיה] 30 93 (pm) 96 יחזקיהו',
  'Lemma Info': {'chapter': '1', 'verses': 1, 'lemmas': [{'lemma': 'יחזקיה'}]},
  'Decoded Entries': [{'Type': 'Variant',
    'Details': [{'Witnesses': [('', '30', ''),
       ('', '93', 'pm'),
       ('', '96', '')]},
     {'Rdg': {'Reading': 'יחזקיהו'}}]}]},
 {'Entry': 'ירבעם בן] 30 + נבט (non voc)',
  'Lemma Info': {'chapter': '1',
   'verses': 1,
   'lemmas': [{'lemma': 'ירבעם'}, {'lemma': 'בן'}]},
  'Decoded Entries': [{'Type': 'Variant',
    'Details': [{'Witnesses': [('', '30', '')]},
     {'Rdg': {'Sigla': '+', 'Reading': 'נבט ', 'Comment': '(non voc)'}}]}]},
 {'Entry': '2 בהושע] 93 (pm) ביהושעIV (similarly PesiqtaR 33 (153b))',
  'Lemma Info': {'chapter': '1', 'verses': 2, 'lemmas': [{'lemma': 'בהו

In [ ]:
# figure out how to present for human QA

In [63]:
#sample entries
full_entries = [
    "Chapter 1",
    "8 k ואמאסאך / q ואמאסך] 30 93 (sm) 96 150 (pm) q IV | 93 (pm) אמסך",
    "8 שָׂך] 30 (pm) 93 (pm) 150 (pm) סךII IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))",
    "6 עוד1] 30 (pm) >I II IV (similarly b. Pesaḥim 87bmss)",
    "6 הדעת1] 93 (pm) 150 (pm) דעתII",
    "14 זעקו] 30 (pm) יזעקו | 150 (pm) >",
    "10 k שעריריה / q שערוריה] 30 (pm) 89 (pm) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) שערורהIV",
    "2 ללבבם] 96 (pm) ללבם | 30 93 150 (pm) בלבבםI",
    "12 עליהם] 96 (pm) להם",
    "איסירם] 30 89 איסרם | 96 (pm) אייסרם, (sm) אייסירים | 150 (pm) אסירים",
    "7 נדמֶה] 93 נדמָה | 30 (pm) + אפרים",
    "10 ואסרם] 96 (pm) יאשרם.. | 150  ..על",
    "8 רֻחמה] 30 לח..",
    "והצגתיה] 93 (pm) + כיום ערומה והצגתיה ",
    "4 דברו] 150 דברים",
    "7–8 ילכו ינקותיו – כיין לבנון] 30 > ",
    "3 כי1] 150 + לא (non voc)",
    "וכְמריו] 93 G-B Msr 34 כֹ",
    "6 בָשנה] G-B Msr 34 בָשְנָה ל ומדׄ מיש ביה בֹשְנָה",
    "k עינתם / q עונֹתם] 30 k, 89 G-B Eb 16 q, 93 96 150 (pm) עונותם",
    "10 לה] 93 (non voc) 96 150 (non voc) + את"
    
]

In [ ]:
#process lemma:
#split into digits and lemmas. then process each separately

In [738]:
[process_full_entry(example) for example in full_entries][-1]#[0]['verses']


({'verses': [10], 'lemmas': [{'lemma': 'ואסרם'}]},
 [[({'Witnesses': [('', '96', 'pm')]},
    {'Reading': {'Reading': 'יאשרם '}},
    {'Cross References': []})],
  [({'Witnesses': [('', '150', '')]},
    {'Reading': {'Reading': 'על'}},
    {'Cross References': []})]])

In [731]:
# old function, didnt take the splitting into commas consideration
# def process_full_entry(text):
#     lemma, part_entry = split_full_entry(text)
#     lemma_dict = lemma_verse_processor(lemma)
# #     if len(lemma_dict['verses'])==0: #get verse from previous entry
# #         lemma_dict['verses'] = 
        
#     #entry_units = split_entry_units # splits on | and ,
#     #for entry in entry_units:
#     decoded_entry = process_entry(part_entry)
#     return lemma_dict, decoded_entry

In [639]:
# processing functions for sub-units of app_entry, for which there is matching lemma and verse data processed above

def remove_and_list_roman_numerals(text): #extract cross-references
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

def custom_split_string(text): #process witnesses
    pattern = re.compile(r'\s?([^\d]*?)?(\d+)\s?\(?([^\)\d]+)?\)?', re.DOTALL|re.UNICODE)    
    #pattern = r'([^,\d]*?)?(\d+)\s?(\(([^\)]+)?\)?)?([\skq]?)+'
    parts = re.findall(pattern, text)
    return parts

def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~]?)                         # Captures special sigla
        \s*
        (?P<Reading>(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*    # Hebrew reading, including 'k', 'q'
                   (?:/\s(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*)?)  # Allows for 'k'/'q' followed by Hebrew, separated by '/'
        \s*
        (?P<Comment>\(.*\))?                     # Captures comments
    """
    
    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v for k, v in match.groupdict().items() if v}

    return parsed_entry

#splitting entry into witnesses and reading (if only one group assign to witnesses)
def split_string(text):
    pattern = re.compile(r'(.*?)?([\+<>]?\s?[kq\u0590-\u05FF]+)(.*)?', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        pattern = re.compile(r'(.*?)([\+<>])(.*)?', re.DOTALL)
        match = pattern.match(text)
        if match:
            return match.groups()  # Returns a tuple with the three parts
        else:
            return text
        return text  # No divider matching the pattern was found


def process_entry(entry):
    clean_entry, cross_references = remove_and_list_roman_numerals(entry)
    split_entry = split_string(clean_entry)
    if type(split_entry) is tuple:
        witnesses = {'Witnesses': custom_split_string(split_entry[0])}
        if len(split_entry)==2:
            reading = parse_reading_entry(split_entry[1])
        else: #there are 3 groups:
            reading = parse_reading_entry(split_entry[1]+split_entry[2])
    else:
        witnesses = {'Witnesses': custom_split_string(split_entry)}
        reading = ''
    return witnesses, {"Reading":reading}, {"Cross References":cross_references}


# for entry in sample_texts:
#     print(f"entry: {entry}")
#     clean_entry, cross_references = remove_and_list_roman_numerals(entry)
#     split_entry = split_string(clean_entry)
#     if type(split_entry) is tuple:
#         witnesses = {'witnesses': custom_split_string(split_entry[0])}
#         reading = parse_reading_entry(split_entry[1])
#         print(f"witnesses: {witnesses}")
#         print(f"reading: {reading}")
#     else:
#         witnesses = {'witnesses': custom_split_string(split_entry)}
#         print(f"witnesses: {witnesses}")
#     print(f"references: {cross_references}")


In [640]:
sample_texts = [
    "96 (non voc)",
    "30 (pm) 93 (pm) 150 (pm) + סךII IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))",
    "93 (non voc) 96 150 (non voc) + את",
    "30 (pm) >",
    "30 + לי (non voc)I II",
    "30 (pm) >I II IV (similarly b. Pesaḥim 87bmss)",
    "93 (pm) ביהושעIV (similarly PesiqtaR 33 (153b))",
    "96 >I II IV",
    "130 k",
    "G-B Msr 34 k ממני / q ממנוIV",
    "93 כד..",
    "150 ..דברים",
    "G-B Eb 94 ותָעָד (understood as \עוד (rather than \עדי))",
    "30 89 (sm) 93 (pm) 150 (non voc) + כיI II IV"
]

In [643]:
process_entry(sample_texts[6])

({'Witnesses': [('', '93', 'pm')]},
 {'Reading': {'Reading': 'ביהושע ',
   'Comment': '(similarly PesiqtaR 33 (153b))'}},
 {'Cross References': ['IV']})

In [638]:
clean_entry, cross_references = remove_and_list_roman_numerals(sample_texts[6])
split_entry = split_string(clean_entry)
if type(split_entry) is tuple:
    witnesses = {'Witnesses': custom_split_string(split_entry[0])}
    if len(split_entry)==2:
        reading = parse_reading_entry(split_entry[1])
    else: #there are 3 groups:
        reading = parse_reading_entry(split_entry[1]+split_entry[2])

reading

{'Reading': 'ביהושע ', 'Comment': '(similarly PesiqtaR 33 (153b))'}

In [462]:
def custom_split_string(text): #process witnesses
    pattern = re.compile(r'\s?([^\d]*?)?(\d+)\s?\(?([^\)\d]+)?\)?', re.DOTALL|re.UNICODE)    
    #pattern = r'([^,\d]*?)?(\d+)\s?(\(([^\)]+)?\)?)?([\skq]?)+'
    parts = re.findall(pattern, text)
    return parts
# "96 (non voc)",
# "30 (pm) 93 (pm) 150 (pm)
test_witness = "30 (pm) 93 150 (pm)"
custom_split_string(test_witness)


[('', '30', 'pm'), ('', '93', ''), ('', '150', 'pm')]

In [623]:
#try parsing single entry app, splitting into witnesses and reading (if only one group assign to witnesses)
def split_string(text):
    pattern = re.compile(r'(.*?)?([\+<>]?\s?[kq\u0590-\u05FF]+)(.*)?', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        pattern = re.compile(r'(.*?)([\+<>])(.*)?', re.DOTALL)
        match = pattern.match(text)
        if match:
            return match.groups()  # Returns a tuple with the three parts
        else:
            return text
        return text  # No divider matching the pattern was found

# Example usage
processed_sample = [split_string(text) for text in sample_texts]

print(processed_sample)


['96 (non voc)', ('30 (pm) 93 (pm) 150 (pm) ', '+ סך', 'II IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'), ('93 (non voc) 96 150 (non voc) ', '+ את', ''), ('30 (pm) ', '>', ''), ('30 ', '+ לי', ' (non voc)I II'), ('30 (pm) ', '>', 'I II IV (similarly b. Pesaḥim 87bmss)'), ('93 (pm)', ' ביהושע', 'IV (similarly PesiqtaR 33 (153b))'), ('96 ', '>', 'I II IV'), ('130', ' k', ''), ('G-B Msr 34', ' k', ' ממני / q ממנוIV'), ('93', ' כד', '..'), ('150 ..', 'דברים', ''), ('G-B Eb 94', ' ותָעָד', ' (understood as \\עוד (rather than \\עדי))'), ('30 89 (sm) 93 (pm) 150 (non voc) ', '+ כי', 'I II IV')]


In [619]:
#function for parsing the reading+comment (assumes witnesses and cross references have been removed)
def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~]?)                         # Captures special sigla
        \s*
        (?P<Reading>(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*    # Hebrew reading, including 'k', 'q'
                   (?:/\s(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*)?)  # Allows for 'k'/'q' followed by Hebrew, separated by '/'
        \s*
        (?P<Comment>\(.*\))?                     # Captures comments
    """
    
    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v for k, v in match.groupdict().items() if v}

    return parsed_entry

# Process the sample reading texts with the refined function
parse_reading_entry = [parse_reading_entry(text) for text in sample_reading_texts]

parse_reading_entry


[{'Reading': 'סך ',
  'Comment': '(See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'},
 {'Sigla': '+', 'Reading': 'את'},
 {'Sigla': '>'},
 {'Sigla': '+', 'Reading': 'לי ', 'Comment': '(non voc)'},
 {'Sigla': '>', 'Comment': '(similarly b. Pesaḥim 87bmss)'},
 {'Reading': 'ביהושע ', 'Comment': '(similarly PesiqtaR 33 (153b))'},
 {},
 {'Reading': 'k'},
 {'Reading': 'k ממני / q ממנו'},
 {'Reading': 'כד..'},
 {'Reading': '..דברים'},
 {'Reading': 'נַחֵם ',
  'Comment': '(taken as infinitive, see Yeivin, Babylonian Vocalization, 1:542)'},
 {'Reading': 'חכֵם ', 'Comment': '(!)'}]

In [587]:
import re

def custom_string_processor(input_string, regex_pattern):
    # Helper function to apply regex and extract groups
    def apply_regex_and_extract(text):
        matches = re.finditer(regex_pattern, text)
        results = []
        for match in matches:
            results.append({
                'witnesses': match.group(1).strip(),
                'reading': match.group(2).strip(),
                'comments': match.group(3).strip() if match.group(3) else ''
            })
        return results

    # Process splits with "|", then ","
    def process_splits(text, delimiter):
        parts = text.split(delimiter)
        processed_parts = []
        for part in parts:
            # Apply regex to each part
            processed = apply_regex_and_extract(part)
            if processed:
                processed_parts.extend(processed)
        return processed_parts

    # Start processing
    processed_result = process_splits(input_string, '|')  # Start with the highest level of split

    return processed_result

# Custom regex pattern as provided
custom_regex = r'^(.*?)([\+<~>]?\s?[\u0590-\u05FF]+.*?)(.*)$'

# Test with the provided sample input
sample_input = "G-B msr. 30 (pm) G-A 89 (sm?) 150 (non voc) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) + שערורהIV II (bla bla (f)) | 150 >"
processed_sample = [custom_string_processor(text, custom_regex) for text in sample_texts]


print(processed_sample)

def split_string(text):
    pattern = re.compile(r'^(.*?)([\+<>]?\s?[\u0590-\u05FF]+.*?)(.*)$', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        return None  # No divider matching the pattern was found

# Example usage
# text = "G-B msr. 30 (pm) G-A 89 (pm?) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) > שערורהIV (bla bla (f))"#"30 89 (sm) 93 (pm) 150 (non voc) + כיI II IV"#"93 (pm) < ביהושעIV (similarly PesiqtaR 33 (153b))"
# split_parts = split_string(text)
# if split_parts:
#     print("witnesses:", split_parts[0])
#     print("reading:", split_parts[1])
#     print("After dividers:", split_parts[2])
# else:
#     print("No dividers found.")



[[], [{'witnesses': '30 (pm) 93 (pm) 150 (pm)', 'reading': '+ סך', 'comments': 'II IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'}], [{'witnesses': '93 (non voc) 96 150 (non voc)', 'reading': '+ את', 'comments': ''}], [], [{'witnesses': '30', 'reading': '+ לי', 'comments': '(non voc)I II'}], [], [{'witnesses': '93 (pm)', 'reading': 'ביהושע', 'comments': 'IV (similarly PesiqtaR 33 (153b))'}], [], [], [{'witnesses': 'G-B Msr 34 k', 'reading': 'ממני', 'comments': '/ q ממנוIV'}], [{'witnesses': '93', 'reading': 'כד', 'comments': '..'}], [{'witnesses': '150 ..', 'reading': 'דברים', 'comments': ''}], [{'witnesses': 'G-B Eb 94', 'reading': 'ותָעָד', 'comments': '(understood as \\עוד (rather than \\עדי))'}], [{'witnesses': '30 89 (sm) 93 (pm) 150 (non voc)', 'reading': '+ כי', 'comments': 'I II IV'}]]


In [37]:

def parse_apparatus_entry(entry):
    """Parse an apparatus entry into lemma(s) and content."""
    parts = entry.split(']')
    lemmas_contents = []
    for part in parts:
        if part.strip():
            lemma, content = part.split('[', 1) if '[' in part else (part, '')
            lemmas_contents.append((lemma.strip(), content.strip()))
    return lemmas_contents

def create_tei_document(apparatus_lines):
    """Create a TEI document from apparatus lines."""
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    TEI = "{%s}" % TEI_NAMESPACE
    NSMAP = {"tei": TEI_NAMESPACE}
    
    tei_root = ET.Element(TEI+"TEI", nsmap=NSMAP)
    tei_header = ET.SubElement(tei_root, TEI+"teiHeader")
    text = ET.SubElement(tei_root, TEI+"text")
    body = ET.SubElement(text, TEI+"body")
    current_chapter = None
    last_verse = None
    
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('Chapter'):
            chapter_number = line.split(' ')[1]
            current_chapter = ET.SubElement(body, TEI+"div", type="chapter", n=chapter_number)
            last_verse = None
            continue
        # Use regex to check if the line starts with a verse number and capture it
        match = re.match(r"^(\d+)\s*(.*)", line)
        if match:
            verse_number, entry = match.groups()
            last_verse = verse_number
        else:
            entry = line
            verse_number = last_verse
        
        if current_chapter is not None and verse_number:
            lemmas_contents = parse_apparatus_entry(entry)
            for lemma, content in lemmas_contents:
                app = ET.SubElement(current_chapter, TEI+"app")
                lem = ET.SubElement(app, TEI+"lem", n=verse_number)
                lem.text = lemma
                if content:
                    rdg = ET.SubElement(app, TEI+"rdg")
                    rdg.text = content

    return ET.ElementTree(tei_root)

def save_tei_file(tree, filename):
    """Save the TEI XML tree to a file."""
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml", short_empty_elements=True)



# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


TEI document has been saved to apparatus_tei.xml.


In [ ]:
####### old stuff

In [47]:
import xml.etree.ElementTree as ET
import re

def create_tei_document(apparatus_lines):
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    TEI = "{%s}" % TEI_NAMESPACE
    
    tei_root = ET.Element(TEI + "TEI", xmlns=TEI_NAMESPACE)
    tei_header = ET.SubElement(tei_root, TEI + "teiHeader")
    text = ET.SubElement(tei_root, TEI + "text")
    body = ET.SubElement(text, TEI + "body")
    current_chapter = None
    last_verse_number = None
    
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('Chapter'):
            chapter_number = line.split(' ')[1].strip()
            current_chapter = ET.SubElement(body, TEI + "div", type="chapter", n=chapter_number)
        else:
            # Attempt to extract verse number and lemma content
            parts = re.match(r"^(\d+)\s*(.*)", line)
            if parts:
                verse_number, remainder = parts.groups()
                last_verse_number = verse_number  # Update last verse number with current
                
                # Further split to separate lemma from variants, if present
                lemma_section, variants_section = remainder.split(']', 1) if ']' in remainder else (remainder, "")
                lemma_section = lemma_section.strip()
                variants_section = variants_section.strip()

                if current_chapter is not None and verse_number:
                    # Create an apparatus entry for the lemma
                    app = ET.SubElement(current_chapter, TEI + "app")
                    lem = ET.SubElement(app, TEI + "lem", n=verse_number)
                    lem.text = lemma_section
                    
                    # Add variant readings if present
                    if variants_section:
                        rdg = ET.SubElement(app, TEI + "rdg")
                        rdg.text = variants_section
            else:
                print(f"Line does not conform to expected format: {line}")

    return ET.ElementTree(tei_root)

def save_tei_file(tree, filename):
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml", short_empty_elements=True)

# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


Line does not conform to expected format: ﻿App III: Hosea
Line does not conform to expected format: יחזקיה] 30 93 (pm) 96 יחזקיהו
Line does not conform to expected format: ירבעם בן] 30 + נבט (non voc)
Line does not conform to expected format: לו] 96 >I  II IV
Line does not conform to expected format: ממלכוּת] 96 ממלכוֹת
Line does not conform to expected format: יזרעאל] 150 ישראל (parall; but 150-Tg: יזרעאל)
Line does not conform to expected format: כי] 93 (pm) + את
Line does not conform to expected format: אוסיף] 150 (pm) >
Line does not conform to expected format: את] 93 (pm) >
Line does not conform to expected format: בסוסים] 96 ובסוסיםI IV
Line does not conform to expected format: אשר2] 96 + לא
Line does not conform to expected format: אחד] 30 (pm) 150 (pm) >
Line does not conform to expected format: והצגתיה] 93 (pm) + כיום ערומה והצגתיה
Line does not conform to expected format: ושתִּה] 150 (pm) ושמתיה
Line does not conform to expected format: כי] 150 (non voc) + כי
Line does not co

In [10]:
import xml.etree.ElementTree as ET
import re


def create_apparatus_entry(verse_number, content, TEI):
    """Create TEI element for an apparatus entry."""
    app = ET.Element(TEI + "app")
    
    # Extract lemma text and the rest (witnesses, variant reading, and comments)
    lemma_text, _, rest = content.partition(']')
    lem = ET.SubElement(app, TEI + "lem")
    lem.text = lemma_text.strip()
    
    # Extract comments
    comments = re.findall(r'\((.*?)\)', rest)
    for comment in comments:
        note = ET.SubElement(app, TEI + "note")
        note.text = comment
    
    # Remove comments from rest for further processing
    rest = re.sub(r'\(.*?\)', '', rest).strip()
    
    # Extract and process witnesses and cross-references
    if rest:
        rdg = ET.SubElement(app, TEI + "rdg")
        witnesses, _, variant_reading = rest.partition(' ')
        if witnesses:
            rdg.set('wit', witnesses.strip())
        if variant_reading:
            rdg.text = variant_reading.strip()
        
        # Extract cross-references, assuming they are indicated by Roman numerals at the start
        cross_refs = re.findall(r'\bI{1,3}V?|\bIV', rest)
        for ref in cross_refs:
            ref_element = ET.SubElement(rdg, TEI + "ref")
            ref_element.set('target', '#' + ref)  # Assuming target IDs are prefixed with '#'
            ref_element.text = "See apparatus entry " + ref
    
    return app

def create_tei_document(apparatus_lines):
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    TEI = "{%s}" % TEI_NAMESPACE
    root = ET.Element(TEI + "TEI", xmlns=TEI_NAMESPACE)
    header = ET.SubElement(root, TEI + "teiHeader")
    text = ET.SubElement(root, TEI + "text")
    body = ET.SubElement(text, TEI + "body")
    div = ET.SubElement(body, TEI + "div")
    
    last_verse_number = None
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        
        # Determine if the line starts with a verse number
        match = re.match(r'^(\d+)', line)
        if match:
            last_verse_number = match.group(1)
            content = line[len(last_verse_number):].strip()
        else:
            content = line
        
        if last_verse_number:
            entry = create_apparatus_entry(last_verse_number, content, TEI)
            div.append(entry)
    
    return ET.ElementTree(root)

def save_tei_file(tree, filename):
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml")


# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


TEI document has been saved to apparatus_tei.xml.


In [232]:
#split entry into witnesses, reading, and comments 

def split_string(text):
    pattern = re.compile(r'^(.*?)([\+<>]?\s?[\u0590-\u05FF]+.*?)(.*)$', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        return None  # No divider matching the pattern was found

# Example usage
text = "G-B msr. 30 (pm) G-A 89 (pm?) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) > שערורהIV (bla bla (f))"#"30 89 (sm) 93 (pm) 150 (non voc) + כיI II IV"#"93 (pm) < ביהושעIV (similarly PesiqtaR 33 (153b))"
split_parts = split_string(text)
if split_parts:
    print("witnesses:", split_parts[0])
    print("reading:", split_parts[1])
    print("After dividers:", split_parts[2])
else:
    print("No dividers found.")


witnesses: G-B msr. 30 (pm) G-A 89 (pm?) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) 
reading: > שערורה
After dividers: IV (bla bla (f))


[('G-B msr. ', '30', 'pm'), ('G-A ', '89', 'pm?'), ('', '150', 'non voc'), ('', '30', 'sm'), ('', '89', 'sm'), ('', '93', 'sm'), ('MS-G ', '150', 'pm')]


In [161]:
#parse comments
import re

def remove_and_list_roman_numerals(text):
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

# Example usage
text = "II IV (See b. R.HaŠanamss 23b, LamR Buber 1:16 (40b))"
result_text, numerals_found = remove_and_list_roman_numerals(text)
print("Modified text:", result_text.strip())
print("Numerals found:", numerals_found)



Modified text: (See b. R.HaŠanamss 23b, LamR Buber 1:16 (40b))
Numerals found: ['II', 'IV']


In [178]:
def process_entry(entry):
    split_parts = split_string(entry)
    if split_parts is None:
        return None
    
    witnesses, reading, comments = split_parts

    structured_entry = {
        'witnesses': [],
        'reading': reading,
        'comments': '',
        'cross_references': []
    }

    for part in custom_split_string(witnesses):
        # Assuming part[1] contains the witness number and part[0], part[2], part[3] contain additional info
        witness_info = {
            'n': part[1],
            'text': f"{part[0]}{part[1]} {part[2].strip()}{part[3]}"
        }
        structured_entry['witnesses'].append(witness_info)

    comments_text, numerals_found = remove_and_list_roman_numerals(comments)
    structured_entry['comments'] = comments_text
    structured_entry['cross_references'] = numerals_found

    return structured_entry

def create_apparatus_entry(verse_number, content, TEI):
    """Create TEI element for an apparatus entry."""
    TEI_ns = {'tei': TEI}  # Define the namespace dictionary if needed
    app = ET.Element(f"{{{TEI}}}app")  # Using namespace in the tag
    
    # Extract lemma text and the rest (witnesses, variant reading, and comments)
    lemma_text, _, rest = content.partition(']')
    lem = ET.SubElement(app, f"{{{TEI}}}lem")
    lem.text = lemma_text.strip('[] ')

    structured_entry = process_entry(rest)
    if not structured_entry:
        return None

    for witness in structured_entry['witnesses']:
        wit_element = ET.SubElement(app, f"{{{TEI}}}wit", {'n': witness['n']})
        wit_element.text = witness['text']
    
    rdg_element = ET.SubElement(app, f"{{{TEI}}}rdg")
    rdg_element.text = structured_entry['reading']

    if structured_entry['comments']:
        comment_element = ET.SubElement(app, f"{{{TEI}}}note")
        comment_element.text = structured_entry['comments']

    for ref in structured_entry['cross_references']:
        ref_element = ET.SubElement(app, f"{{{TEI}}}ref")
        ref_element.text = ref

    return app


In [181]:
import xml.etree.ElementTree as ET
import re


def create_tei_document(apparatus_lines):
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    ET.register_namespace('', TEI_NAMESPACE)  # Register the default namespace

    # Create the root element without redundantly specifying the xmlns attribute
    root = ET.Element("{%s}TEI" % TEI_NAMESPACE)
    header = ET.SubElement(root, "{%s}teiHeader" % TEI_NAMESPACE)
    text = ET.SubElement(root, "{%s}text" % TEI_NAMESPACE)
    body = ET.SubElement(text, "{%s}body" % TEI_NAMESPACE)
    div = ET.SubElement(body, "{%s}div" % TEI_NAMESPACE)
    
    last_verse_number = None
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        
        # Determine if the line starts with a verse number
        match = re.match(r'^(\d+)', line)
        if match:
            last_verse_number = match.group(1)
            content = line[len(last_verse_number):].strip()
        else:
            content = line
        
        if last_verse_number:
            entry = create_apparatus_entry(last_verse_number, content, TEI_NAMESPACE)
            if entry is not None:  # Ensure entry creation was successful
                div.append(entry)
    
    return ET.ElementTree(root)

def save_tei_file(tree, filename):
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml")


# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


TEI document has been saved to apparatus_tei.xml.


In [227]:
import re

def split_string(text):
    pattern = re.compile(r'^(.*?)([\+<~>]?\s?[\u0590-\u05FF]+.*?)(.*)$', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()
    else:
        return None

def custom_split_string(text):
    pattern = re.compile(r'([^,\d]*?)?(\d+)\s?(\([^\)]+\)?)?([\skq]*)?', re.DOTALL|re.UNICODE)
    parts = re.findall(pattern, text)
    return parts

def remove_and_list_roman_numerals(text):
    pattern = r'([I]*[V]?)'
    found_numerals = re.findall(pattern, text)
    found_numerals = [numeral for numeral in found_numerals if numeral]
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

# def process_entry(entry):
#     split_parts = split_string(entry)
#     if split_parts is None:
#         return "Unable to process entry: No valid dividers found."
    
#     witnesses, reading, comments = split_parts

#     witness_entries = []
#     for part in custom_split_string(witnesses):
#         witness_entry = f'<witness n="{part[1]}">{part[0]}{part[1]} {part[2].strip()}{part[3]}</witness>'
#         witness_entries.append(witness_entry)
#     witnesses_tagged = "\n".join(witness_entries)

#     reading_tagged = f'<reading>{reading}</reading>'

#     comments_text, numerals_found = remove_and_list_roman_numerals(comments)
#     comments_tagged = f'<comment>{comments_text}</comment>'
#     cross_references = "\n".join([f'<ref>{numeral}</ref>' for numeral in numerals_found])

#     # Combine all parts, placing cross_references outside the comment
#     tei_entry = f"{witnesses_tagged}\n{reading_tagged}\n{comments_tagged}\n{cross_references}"
#     return tei_entry

# Example usage
entry ="G-B msr. 30 (pm) G-A 89 (pm?) 150 (non voc) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) > שערורהIV II (bla bla (f))"
processed_entry = process_entry(entry)
print(processed_entry)


{'witnesses': [{'n': '30', 'text': 'G-B msr. 30 (pm) '}, {'n': '89', 'text': 'G-A 89 (pm?) '}, {'n': '150', 'text': '150 (non voc) k'}, {'n': '30', 'text': ' 30 (sm) '}, {'n': '89', 'text': '89 (sm) '}, {'n': '93', 'text': '93 (sm) '}, {'n': '96', 'text': '96 '}, {'n': '150', 'text': '150 (pm) q'}, {'n': '93', 'text': ' 93 (pm) '}], 'reading': '> שערורה', 'comments': '  (bla bla (f))', 'cross_references': ['IV', 'II']}


In [ ]:
#### 

In [ ]:
[' ', '"', '$', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', 'E', 'I', 'T', '_', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', '֑', '֔', '֕', '֖', '֗', '֙', '֛', '֜', '֞', '֣', '֤', '֥', '֨', '֩', 'ְ', 'ֱ', 'ֲ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֻ', 'ּ', 'ֽ', '׀', 'ׁ', 'ׂ']

In [17]:
#old function. didnt handle the "E-G misr" etc.
def compile_and_display_entries(entries):
    previous_verse = None
    compiled_results = []

    for entry in entries:
        lemma_dict, decoded_entries, used_verse = process_full_entry(entry, previous_verse)
        previous_verse = used_verse

        # Combine the lemma_dict and decoded_entries for display
        entry_result = {
            'Lemma': lemma_dict,
            'Decoded Entries': decoded_entries
        }
        compiled_results.append(entry_result)

    # Display the results in a structured format
    for result in compiled_results:
        print("Lemma:")
        for key, value in result['Lemma'].items():
            print(f"  {key}: {value}")
        print("Decoded Entries:")
        last_witness_second_group = None
        for i, decoded_entries_list in enumerate(result['Decoded Entries']):
            for j, decoded_entry in enumerate(decoded_entries_list):
                variant_type = "Variant" if j == 0 else "Related Variant"
                print(f"  {variant_type}:")
                for item in decoded_entry:
                    if isinstance(item, dict) and 'Witnesses' in item:
                        # Update witnesses with last non-empty second group if needed
                        updated_witnesses = []
                        for witness in item['Witnesses']:
                            if witness[1] == '' and last_witness_second_group:
                                updated_witnesses.append((witness[0], last_witness_second_group, witness[2]))
                            else:
                                updated_witnesses.append(witness)
                                if witness[1]:
                                    last_witness_second_group = witness[1]
                        item['Witnesses'] = updated_witnesses
                    if isinstance(item, dict):
                        for key, value in item.items():
                            print(f"    {key}: {value}")
                    else:
                        print(f"    {item}")
        print("-" * 50)  # Separator line

# Example usage
compile_and_display_entries(full_entries)


Lemma:
  verses: [8]
  lemmas: [{'lemma': 'ואמאסאך', 'k': True}, {'lemma': 'ואמאסך', 'q': True}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', ''), ('', '93', 'sm'), ('', '96', ''), ('', '150', 'pm')]
    Reading: {'Reading': 'q  '}
    Cross References: ['IV']
  Variant:
    Witnesses: [('', '93', 'pm')]
    Reading: {'Reading': 'אמסך'}
--------------------------------------------------
Lemma:
  verses: [8]
  lemmas: [{'lemma': 'שָׂך'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', 'pm'), ('', '93', 'pm'), ('', '150', 'pm')]
    Reading: {'Reading': 'סך  ', 'Comment': '(See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'}
    Cross References: ['II', 'IV']
--------------------------------------------------
Lemma:
  verses: [6]
  lemmas: [{'lemma': 'עוד', 'number': '1'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', 'pm')]
    Reading: {'Sigla': '>', 'Comment': '(similarly b. Pesaḥim 87bmss)'}
    Cross References: ['I', 'II', 'IV']
----------------------------

In [39]:
#old processing that doesnt include chapter extraction
def post_process_witnesses(witnesses):
    # Define the set of specific values for "x"
    specific_values = {"G-B Eb ", "G-B Kb ", "G-B Msr "}  # Replace with the actual values

    # Iterate over the witnesses, except for the last one
    for i in range(len(witnesses) - 1):
        z, y, x = witnesses[i]

        # Check if "x" is one of the specific values
        if x in specific_values:
            # Remove "x" from the current tuple and prepend it to the "z" of the next witness
            witnesses[i] = (z, y, None)
            next_z, next_y, next_x = witnesses[i + 1]
            witnesses[i + 1] = (x + next_z, next_y, next_x)

    # Remove the first tuple if it becomes empty
    if witnesses and witnesses[0] == (None, None, None):
        witnesses.pop(0)

    return witnesses

# Example usage
def compile_and_display_entries(entries):
    previous_verse = None
    compiled_results = []

    for entry in entries:
        lemma_dict, decoded_entries, used_verse = process_full_entry(entry, previous_verse)
        previous_verse = used_verse

        # Combine the lemma_dict and decoded_entries for display
        entry_result = {
            'Lemma': lemma_dict,
            'Decoded Entries': decoded_entries
        }
        compiled_results.append(entry_result)

    # Display the results in a structured format
    for result in compiled_results:
        print("Lemma:")
        for key, value in result['Lemma'].items():
            print(f"  {key}: {value}")
        print("Decoded Entries:")
        last_witness_second_group = None
        for i, decoded_entries_list in enumerate(result['Decoded Entries']):
            for j, decoded_entry in enumerate(decoded_entries_list):
                variant_type = "Variant" if j == 0 else "Related Variant"
                print(f"  {variant_type}:")
                for item in decoded_entry:
                    if isinstance(item, dict) and 'Witnesses' in item:
                        # Update witnesses with last non-empty second group if needed
                        updated_witnesses = []
                        for witness in item['Witnesses']:
                            if witness[1] == '' and last_witness_second_group:
                                updated_witnesses.append((witness[0], last_witness_second_group, witness[2]))
                            else:
                                updated_witnesses.append(witness)
                                if witness[1]:
                                    last_witness_second_group = witness[1]
                        # Apply post-processing to witnesses
                        item['Witnesses'] = post_process_witnesses(updated_witnesses)
                    if isinstance(item, dict):
                        for key, value in item.items():
                            print(f"    {key}: {value}")
                    else:
                        print(f"    {item}")
        print("-" * 50)  # Separator line
    return compiled_results
# Example usage
test = compile_and_display_entries(full_entries)


Lemma:
  verses: [8]
  lemmas: [{'lemma': 'ואמאסאך', 'k': True}, {'lemma': 'ואמאסך', 'q': True}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', ''), ('', '93', 'sm'), ('', '96', ''), ('', '150', 'pm')]
    Reading: {'Reading': 'q  '}
    Cross References: ['IV']
  Variant:
    Witnesses: [('', '93', 'pm')]
    Reading: {'Reading': 'אמסך'}
--------------------------------------------------
Lemma:
  verses: [8]
  lemmas: [{'lemma': 'שָׂך'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', 'pm'), ('', '93', 'pm'), ('', '150', 'pm')]
    Reading: {'Reading': 'סך  ', 'Comment': '(See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'}
    Cross References: ['II', 'IV']
--------------------------------------------------
Lemma:
  verses: [6]
  lemmas: [{'lemma': 'עוד', 'number': '1'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', 'pm')]
    Reading: {'Sigla': '>', 'Comment': '(similarly b. Pesaḥim 87bmss)'}
    Cross References: ['I', 'II', 'IV']
----------------------------